# First, do NN prediction. Save it in var "yp", delete all but that and "df3"

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import os, warnings, gc, random, sys, math, joblib
from scipy.signal import savgol_filter
from pathlib import Path
from time import time
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import QuantileTransformer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay, CosineDecayRestarts
from tensorflow.keras.regularizers import L2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import Layer
from tensorflow.keras.layers import Input, Dense, Softmax, Dropout, LSTM, Bidirectional, Concatenate, Embedding
from tensorflow.keras.utils import Sequence


warnings.filterwarnings('ignore')
t0 = time()

data_dir = '/kaggle/input/MABe-mouse-behavior-detection'
seed = 13

model_dir = '/kaggle/input/nn-data-mabe/nn/'
nn_mode = 'submit' # 'train'/'validate'/'submit'

folds = 5

2025-12-15 16:04:18.306639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765814658.515408      28 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765814658.577958      28 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


NN functions

In [2]:
# hardcoded di of all actions
# single actions are: 0, 10, 21-24, 26, 29, 30, 32 (11 out of 34)
N_ACT = 33
N_LAB = 17 # number of labs; last 3 are combined

# the 'ejaculate' label was supposed to be removed; It does not appear in the test set so can be safely ignored. - done. Was 23.
# drop 'dominancemount'=15, 'genitalgroom'=18 - confirmed by LB30 - incr of 2 points.
# drop 'disengage'=10 - it is only in CRIM13 - confirm by LB38 - decr of 3 points!
action_di = {'rear': 0, 'avoid': 1, 'attack': 2, 'approach': 3, 'chase': 4, 'submit': 5, 'chaseattack': 6, 'shepherd': 7, 'sniff': 8, 'mount': 9
    , 'disengage': -1, 'selfgroom': 10, 'sniffgenital': 11, 'sniffface': 12, 'sniffbody': 13, 'dominancemount': -1, 'attemptmount': 14, 'intromit': 15
    , 'genitalgroom': -1, 'reciprocalsniff': 16, 'escape': 17, 'dominance': 18, 'allogroom': 19, 'ejaculate': -1, 'defend': 20, 'climb': 21, 'dig': 22
    , 'rest': 23, 'run': 24, 'dominancegroom': 25, 'freeze': 26, 'flinch': 27, 'follow': 28, 'exploreobject': 29, 'biteobject': 30, 'tussle': 31
    , 'huddle': 32}
action_di2 = {0: 'rear', 1: 'avoid', 2: 'attack', 3: 'approach', 4: 'chase', 5: 'submit', 6: 'chaseattack', 7: 'shepherd', 8: 'sniff', 9: 'mount'
    , 10: 'selfgroom', 11: 'sniffgenital', 12: 'sniffface', 13: 'sniffbody', 14: 'attemptmount', 15: 'intromit'
    , 16: 'reciprocalsniff', 17: 'escape', 18: 'dominance', 19: 'allogroom', 20: 'defend', 21: 'climb', 22: 'dig'
    , 23: 'rest', 24: 'run', 25: 'dominancegroom', 26: 'freeze', 27: 'flinch', 28: 'follow', 29: 'exploreobject', 30: 'biteobject', 31: 'tussle'
    , 32: 'huddle'}

# body_parts_tracked
# remap 'head' to 'nose' - they never occur together, and are close on the body. Head is only for lab 'GroovyShrew'=5, id=[555-571]
# drop all past 5. Always use 0123, then either 4 or (5+6)/2, or fill-in with average of all of them. Then add 7=f
bp_di = {'tail_base':0,'ear_right':1,'ear_left':2,'nose':3,'head':3,'body_center':4,'hip_right':5,'hip_left':6,'tail_tip':7,
    'neck':8,'lateral_right':9,'lateral_left':10,'tail_midpoint':11,'tail_middle_1':13,'spine_2':14,'spine_1':15,'tail_middle_2':16,'headpiece_topfrontright':17,
    'headpiece_topbackright':18,'headpiece_topfrontleft':19,'headpiece_topbackleft':20,'headpiece_bottomfrontright':21,'headpiece_bottombackright':22,
    'headpiece_bottombackleft':23,'headpiece_bottomfrontleft':24}

# lab text to numeric mapping
lab_di = {'AdaptableSnail': 0, 'BoisterousParrot': 1, 'CautiousGiraffe': 2, 'DeliriousFly': 3, 'ElegantMink': 4, 'GroovyShrew': 5
          , 'InvincibleJellyfish': 6, 'JovialSwallow': 7, 'LyricalHare': 8, 'NiftyGoldfinch': 9, 'PleasantMeerkat': 10
          , 'ReflectiveManatee': 11, 'SparklingTapir': 12, 'TranquilPanther': 13, 'UppityFerret': 14 # real labs: 0-14
          , 'CRIM13': 15, 'CalMS21_supplemental': 16, 'CalMS21_task1': 17, 'CalMS21_task2': 18} # additional training labs only

norm_di = {} # put values here if training is not run





# combine all 5 body parts ***********************************************************************************
def body_parts(det_df):
    # combine first 4 normally
    det_df2 = (det_df
          .filter(pl.col('bodypart') == 0)
          .select(['video_frame', 'mouse_id', 'x', 'y'])
          .rename({'x': 'x_a', 'y': 'y_a'})
          .join(
              det_df
              .filter(pl.col('bodypart') == 1)
              .select(['video_frame', 'mouse_id', 'x', 'y'])
              .rename({'x': 'x_b', 'y': 'y_b'})
              , how='full', on=['mouse_id', 'video_frame'], coalesce=True
              )
          .join(
              det_df
              .filter(pl.col('bodypart') == 2)
              .select(['video_frame', 'mouse_id', 'x', 'y'])
              .rename({'x': 'x_c', 'y': 'y_c'})
              , how='full', on=['mouse_id', 'video_frame'], coalesce=True
              )
          .join(
              det_df
              .filter(pl.col('bodypart') == 3)
              .select(['video_frame', 'mouse_id', 'x', 'y'])
              .rename({'x': 'x_d', 'y': 'y_d'})
              , how='full', on=['mouse_id', 'video_frame'], coalesce=True
              )
          )
    
    # add 5th one, one of 3 ways
    if det_df.filter(pl.col('bodypart') == 4).shape[0] > 0: # use body center 'as is' - most labs
        det_df2 = (det_df2
            .join(
                det_df
                .filter(pl.col('bodypart') == 4)
                .select(['video_frame', 'mouse_id', 'x', 'y'])
                .rename({'x': 'x_e', 'y': 'y_e'})
                , how='full', on=['mouse_id', 'video_frame'], coalesce=True
                )
            )
    elif det_df.filter(pl.col('bodypart') == 5).shape[0] > 0: # use middle of hips (5+6) - labs 2,4,6,7,13
        det_df3 = (det_df
            .filter(pl.col('bodypart') == 5)
            .select(['video_frame', 'mouse_id', 'x', 'y'])
            .rename({'x': 'x_1', 'y': 'y_1'})
            .join(
                det_df
                .filter(pl.col('bodypart') == 6)
                .select(['video_frame', 'mouse_id', 'x', 'y'])
                .rename({'x': 'x_2', 'y': 'y_2'})
                , how='full', on=['mouse_id', 'video_frame'], coalesce=True
                )
            )
        det_df3 = (det_df3
            .with_columns(x_e = ((pl.col('x_1') + pl.col('x_2')) / 2).cast(pl.Float32))
            .with_columns(y_e = ((pl.col('y_1') + pl.col('y_2')) / 2).cast(pl.Float32))
            .select(['video_frame', 'mouse_id', 'x_e', 'y_e'])
            )
        det_df2 = (det_df2
            .join(
                det_df3
                , how='full', on=['mouse_id', 'video_frame'], coalesce=True
                )
            )
    else: # use average* of first 4 body parts - labs 5, 8
        det_df2 = (det_df2
            .with_columns(x_e = ((pl.col('x_a') + pl.col('x_b') + pl.col('x_c') + pl.col('x_d')) / 4).cast(pl.Float32))
            .with_columns(y_e = ((pl.col('y_a') + pl.col('y_b') + pl.col('y_c') + pl.col('y_d')) / 4).cast(pl.Float32))
            )
        
    # add tail tip
    if det_df.filter(pl.col('bodypart') == 7).shape[0] > 0: # use tail tip 'as is' - 15% of frames, 5 labs
        det_df2 = (det_df2
            .join(
                det_df
                .filter(pl.col('bodypart') == 7)
                .select(['video_frame', 'mouse_id', 'x', 'y'])
                .rename({'x': 'x_f', 'y': 'y_f'})
                , how='full', on=['mouse_id', 'video_frame'], coalesce=True
                )
            )
    else: # use -1 to indicate missing tail. Handle it later.
        det_df2 = (det_df2
            .with_columns(x_f = (pl.col('x_a') * 0 - 1).cast(pl.Float32))
            .with_columns(y_f = (pl.col('y_a') * 0 - 1).cast(pl.Float32))
            )
    return det_df2





# read data ***********************************************************************************
def read_data(train = 0):
    # meta
    if train:
        dfm = pd.read_csv(data_dir + '/train.csv')
    else:
        dfm = pd.read_csv(data_dir + '/test.csv')

    # details and annotations
    dfa = pd.DataFrame()
    df = pl.DataFrame()
    i, j = 0, 0
    lab_id0 = ''
    id_di = {}
    id_di2 = {}
    for _, row in dfm.iterrows():
        lab_id = row['lab_id'] # 19 labs
        video_id = row['video_id'] # many vids
        if train:
            ann_path = data_dir + '/train_annotation/' + lab_id + '/' + str(video_id) + '.parquet'
            det_path = data_dir + '/train_tracking/' + lab_id + '/' + str(video_id) + '.parquet'
        else:
            det_path = data_dir + '/test_tracking/' + lab_id + '/' + str(video_id) + '.parquet'
        # skip training data without annotations
        if os.path.exists(det_path) and (train == 0 or os.path.exists(ann_path)):
            # count videos from the same lab
            if lab_id0 == lab_id:
                j += 1
            else:
                j = 0
            lab_id0 = lab_id

            if train:
                ann_df = pd.read_parquet(ann_path)
                ann_df['agent_id'] -= 1 # now starts from 0
                ann_df['target_id'] -= 1 # now starts from 0
                ann_df['id'] = i # add id
                ann_df['id'] = ann_df['id'].astype('int16') # reduce size
                dfa = pd.concat([dfa, ann_df], axis=0, ignore_index=True)
                
            # polars code for details
            multx = np.maximum(.42, np.minimum(4.5, 30 * row['arena_width_cm'] / row['video_width_pix']))
            multy = np.maximum(.42, np.minimum(4.5, 30 * row['arena_height_cm'] / row['video_height_pix']))
            det_df = (pl.read_parquet(det_path)
                      .with_columns(bodypart = pl.col('bodypart').map_elements(lambda x: bp_di[x], return_dtype=pl.Int8)) # map bodypart
                      .filter(pl.col('bodypart') < 8) # drop all but 8 - incl tail_tip=7
                      .with_columns(x = (pl.col('x') * multx).cast(pl.Float32)) # scale coordinates by lab [lab_scale[lab_id]]
                      .with_columns(y = (pl.col('y') * multy).cast(pl.Float32))
                      )
            det_df = body_parts(det_df) # combine all body parts - one video at a time. Before adding 'id'
            det_df = (det_df
                      .with_columns(id = i) # add id
                      .with_columns(video_frame = pl.col('video_frame').cast(pl.Int32), id = pl.col('id').cast(pl.Int16)) # cast
                      )
            df = pl.concat([df, det_df], how='vertical') # concat

            print('    ', i, j, lab_id + '/' + str(video_id), det_df.shape[0])
            id_di[i] = [lab_id, video_id, lab_di[lab_id]] # decode: lab[text], video, lab[numeric]
            id_di2[lab_id + '/' + str(video_id)] = i
            i += 1 # incr id

    print('    finished reading files', int(time() - t0), 'sec')
    df = df.to_pandas()

    # drop bad videos
    vid = set(dfm['video_id'].loc[(dfm['frames_per_second']==25) & (dfm['lab_id']=='AdaptableSnail')]) # list of 'bad' video_id values
    vid = [id_di2['AdaptableSnail/' + str(video_id)] for video_id in vid if 'AdaptableSnail/' + str(video_id) in id_di2] # list of 'bad' id values. In train it is [1, 8, 9, 13]
    df = df.loc[df['id'].map(lambda x: x not in vid)].reset_index(drop=True) # drop them
    
    df['mouse_id'] -= 1 # now starts from 0
    df = df.sort_values(by=['mouse_id', 'id', 'video_frame']).reset_index(drop=True) # sort, JIC

    # determine the average size of each mouse - before interpolation. From a/b/c points only
    df['s'] = np.sqrt((df['x_a'] - df['x_b'] / 2 - df['x_c'] / 2)**2+(df['y_a'] - df['y_b'] / 2 - df['y_c'] / 2)**2) # use a+b+c only
    a = df.groupby(['mouse_id', 'id'])['s'].agg(['median', 'mean', 'std', 'max', 'min', 'size']).reset_index()
    df.drop('s', axis=1, inplace=True)
    a['median'] = np.maximum(30, a['median'].fillna(a['median'].mean())) # use mean when missing, floor at 30
    m_size_di = dict(zip(list(a['id'] * 4 + a['mouse_id']), list(np.round(a['median'], 1)))) # key: id * 4 + mouse_id

    # fill missing values
    print('    interpolating missing values...', int(time() - t0), 'sec')
    df.interpolate(method='polynomial', order=1, inplace=True, axis=0) # slow
    df = df.fillna(method='ffill') # after interpolation some values are still missing
    df = df.fillna(method='bfill') # after ffill some values are still missing
    
    if train:
        dfa0 = dfa.copy() # save for CV
        # format solution
        dfa0['id2'] = dfa0['id'].map(id_di)
        dfa0['lab_id'] = dfa0['id2'].str[0]
        dfa0['video_id'] = dfa0['id2'].str[1]
        dfa0.drop('id2', axis=1, inplace=True)
        dfa0['agent_id'] = dfa0['agent_id'].apply(lambda x: 'mouse' + str(x+1))
        dfa0['target_id'] = dfa0['target_id'].apply(lambda x: 'mouse' + str(x+1))
        dfa0 = dfa0.merge(dfm[['lab_id', 'video_id', 'behaviors_labeled']], how='left', on=['lab_id', 'video_id'])
        # relabel self-actions
        dfa0['target_id'].loc[dfa0['target_id'] == dfa0['agent_id']] = 'self'
        # drop exlcuded actions from official solution
        for a in ['dominancemount', 'genitalgroom', 'ejaculate', 'disengage']:
            dfa0 = dfa0.loc[dfa0['action'] != a]
        return df, dfm, dfa, dfa0, id_di, id_di2, m_size_di
    else:
        return df, dfm, id_di, id_di2, m_size_di





# format into mouse pairs ***********************************************************************************
def prepare_pairs():
    global df
    df = pl.from_pandas(df)
    df3 = pl.DataFrame()
    for m1, m2 in zip([0, 0, 0, 1, 1, 2, 1, 2, 3, 2, 3, 3, 0, 1, 2, 3], [1, 2, 3, 2, 3, 3, 0, 0, 0, 1, 1, 2, 0, 1, 2, 3]):
        dft = (df.filter(pl.col('mouse_id') == m1)
              .select(['id', 'video_frame', 'x_a', 'y_a', 'x_b', 'y_b', 'x_c', 'y_c', 'x_d', 'y_d', 'x_e', 'y_e', 'x_f', 'y_f'])
              .rename({'x_a': 'x_1_a', 'y_a': 'y_1_a', 'x_b': 'x_1_b', 'y_b': 'y_1_b', 'x_c': 'x_1_c', 'y_c': 'y_1_c', 'x_d': 'x_1_d', 'y_d': 'y_1_d', 'x_e': 'x_1_e', 'y_e': 'y_1_e', 'x_f': 'x_1_f', 'y_f': 'y_1_f'})
              .with_columns(mid = m1 * 10 + m2) # add mid
              .with_columns(mid = pl.col('mid').cast(pl.Int8)) # cast
              .join(
                  df
                  .filter(pl.col('mouse_id') == m2)
                  .select(['id', 'video_frame', 'x_a', 'y_a', 'x_b', 'y_b', 'x_c', 'y_c', 'x_d', 'y_d', 'x_e', 'y_e', 'x_f', 'y_f'])
                  .rename({'x_a': 'x_2_a', 'y_a': 'y_2_a', 'x_b': 'x_2_b', 'y_b': 'y_2_b', 'x_c': 'x_2_c', 'y_c': 'y_2_c', 'x_d': 'x_2_d', 'y_d': 'y_2_d', 'x_e': 'x_2_e', 'y_e': 'y_2_e', 'x_f': 'x_2_f', 'y_f': 'y_2_f'})
                  , how='inner', on=['id', 'video_frame']
                  )
               )
        df3 = pl.concat([df3, dft], how='vertical') # concat
    del df, dft
    gc.collect()
    df3 = df3.to_pandas()
    df3 = df3[['id', 'mid', 'video_frame', 'x_1_a', 'x_1_b', 'x_1_c', 'x_1_d', 'x_1_e', 'x_1_f', 'x_2_a', 'x_2_b', 'x_2_c', 'x_2_d', 'x_2_e', 'x_2_f',
               'y_1_a', 'y_1_b', 'y_1_c', 'y_1_d', 'y_1_e', 'y_1_f', 'y_2_a', 'y_2_b', 'y_2_c', 'y_2_d', 'y_2_e', 'y_2_f']] # reorder
    df3 = df3.sort_values(by=['id', 'mid', 'video_frame']).reset_index(drop=True) # need this order - frame last!
    return df3





# construct 'behaviors_labeled' mask ***********************************************************************************
def prepare_mask(df3):
    global dfm
    # format dfm
    dfm['id'] = dfm['lab_id'] + '/' + dfm['video_id'].astype('str')
    dfm['id'] = dfm['id'].map(id_di2)
    dfm = dfm.loc[~dfm['behaviors_labeled'].isna()].reset_index(drop=True) # drop empty
    # replace actions with their num codes
    dfm['behaviors_labeled'] = dfm['behaviors_labeled'].apply(lambda x: x.replace("'","")) # remove "'"
    a = list(action_di.keys())
    a.sort(key=lambda s: -len(s)) # sort actions by decr length, to enforce exact match
    for k in a:
        dfm['behaviors_labeled'] = dfm['behaviors_labeled'].apply(lambda x: x.replace(k, str(action_di[k])))
    # replace mice combos with their codes
    for m1 in [1,2,3,4]:
        t = str(m1 * 10 + m1 - 11)
        dfm['behaviors_labeled'] = dfm['behaviors_labeled'].apply(lambda x: x.replace('mouse'+str(m1)+',self', t))
        for m2 in [1,2,3,4]:
            t = str(m1 * 10 + m2 - 11) # is this in correct order? Confirmed, it is.
            dfm['behaviors_labeled'] = dfm['behaviors_labeled'].apply(lambda x: x.replace('mouse'+str(m1)+',mouse'+str(m2), t))

    # first, construct mask for id/mid keys only
    dfm = dfm.loc[~dfm['id'].isna()].reset_index(drop=True) # drop missing id
    res = []
    a = list(dfm['behaviors_labeled'])
    for i in range(dfm.shape[0]):
        iid = int(dfm['id'].iloc[i])
        b = a[i].split()
        b = [x.replace('[', '').replace(']', '').replace('"', '') for x in b]
        di = {}
        for bi in b:
            c = bi.split(',')
            if int(c[0]) not in di:
                di[int(c[0])] = c[1]
            else:
                di[int(c[0])] = di[int(c[0])] + ',' + c[1]
        for k in di.keys():
            res.append([iid, k, di[k]])
    dfm2 = pd.DataFrame(res)
    dfm2.columns = ['id', 'mid', 'actions']

    # second, expand all actions
    for i in range(N_ACT):
        dfm2['a'+str(i)] = dfm2['actions'].apply(lambda x: str(i) in x.split(',')).astype('int8')
    dfm2.drop('actions', axis=1, inplace=True)
    dfm2 = dfm2.loc[dfm2.iloc[:,2:].values.sum(-1) > 0].reset_index(drop=True) # drop records with no action: 1440->1423; happens when the only action is excluded

    # third, merge with full data. 50% are NAs, but all of them are for records with action=-1, so they can be dropped
    df_mask = df3[['id', 'mid']].merge(dfm2[['id', 'mid', 'a0']], how='left', on=['id', 'mid'])
    idx = df_mask['a0'].isna()
    print('    proportion without behaviors_labeled, drop', np.round(df_mask['a0'].isna().mean(), 3))

    # drop data for not labeled actions
    df3 = df3.loc[~idx].reset_index(drop=True)
    df_mask = df_mask.loc[~idx].reset_index(drop=True)

    # now merge with full data; since nothing is missing, int8 stays int8 and greatly reduced RAM usage
    del df_mask
    gc.collect()
    df_mask = df3[['id', 'mid']].merge(dfm2, how='left', on=['id', 'mid'])
    mask = df_mask.iloc[:,2:].values.astype(np.int8) # skip id, mid
    del idx, a, dfm2, res, di, df_mask
    gc.collect()
    # add leading 'no action'
    mask = np.concatenate((mask[:,0].reshape(-1,1), mask), axis=1)
    mask[:,0] = 1 # 'no action' is always labeled
    return df3, mask





def fit_ellipse(df, cols=['x_a','x_b','x_c','x_d','x_e','y_a','y_b','y_c','y_d','y_e']): # returns major/minor axis based on 5 points
    data = df[cols].values.reshape(-1,2,5)
    data = data - data.mean(-1).reshape(-1,2,1) # center
    covariance = np.zeros([data.shape[0], 2, 2], dtype=np.float32)
    covariance[:,0,0] = data[:,0,:].var(-1)
    covariance[:,1,1] = data[:,1,:].var(-1)
    covariance[:,0,1] = (data[:,0,:] * data[:,1,:]).mean(-1)
    covariance[:,1,0] = covariance[:,0,1]
    D = np.linalg.svd(covariance, full_matrices=True, compute_uv=False, hermitian=True)
    a, b = np.sqrt(D[:,0]), np.sqrt(D[:,1])
    # clip
    a = np.clip(a, 3, 500)
    b = np.clip(b, 1, 100)
    return a, b





# FE
def FE(df3):
    global mask, dfm

    # single
    df3['single'] = (((df3['mid']//10) == (df3['mid']%10)) * 1).astype('int8')

    # mid: 0/11/22/33=self; inverse: 10/20/21/30/31/32  1/2/3/12/13/23/10/20/21/30/31/32
    df3['inverse'] = (((df3['mid']//10) > (df3['mid']%10)) * 1).astype('int8')

    # lab code. Split into 4. Use dict to make sure it works on sub!!!
    l = np.minimum(N_LAB -1, df3['id'].map(id_di).str[2].astype(np.int8)) # 0-16 - collapse last 3 labs
    df3['l1'] = (l%2).astype(np.int8) # 0-1
    df3['l2'] = ((l//2)%2).astype(np.int8) # 0-1
    df3['l3'] = ((l//4)%2).astype(np.int8) # 0-1
    df3['l4'] = ((l//8)%2).astype(np.int8) # 0-1
    df3['l5'] = (l//16).astype(np.int8) # 0-1. 1 only for excluded labs in training
    del l
    gc.collect()

    # add sex of both mice; missing sex = 80% male. So values are 0/1/.2 - pretty much binary, leave it 'as is'
    # first, create mapping of id + mouse_id to sex
    dfm['id'] = dfm['id'].astype('int16')
    m_sex_di = dict(zip(list(dfm['id'] * 4 + 0), list(dfm['mouse1_sex'].map({'male':0, 'female':1}).fillna(.2)))) # key: id * 4 + mouse_id
    m_sex_di = m_sex_di | dict(zip(list(dfm['id'] * 4 + 1), list(dfm['mouse2_sex'].map({'male':0, 'female':1}).fillna(.2)))) # key: id * 4 + mouse_id
    m_sex_di = m_sex_di | dict(zip(list(dfm['id'] * 4 + 2), list(dfm['mouse3_sex'].map({'male':0, 'female':1}).fillna(.2)))) # key: id * 4 + mouse_id
    m_sex_di = m_sex_di | dict(zip(list(dfm['id'] * 4 + 3), list(dfm['mouse4_sex'].map({'male':0, 'female':1}).fillna(.2)))) # key: id * 4 + mouse_id
    df3['sex1'] = (df3['id'] * 4 + df3['mid'] // 10).map(m_sex_di).astype('float32')
    df3['sex2'] = (df3['id'] * 4 + df3['mid'] % 10).map(m_sex_di).astype('float32')

    # add color of both mice; missing(=brown) color = .5. So values are 0/1/.5 - pretty much binary, leave it 'as is'
    # first, create mapping of id + mouse_id to color
    m_c_di = dict(zip(list(dfm['id'] * 4 + 0), list(dfm['mouse1_color'].map({'black':0, 'white':1}).fillna(.5)))) # key: id * 4 + mouse_id
    m_c_di = m_c_di | dict(zip(list(dfm['id'] * 4 + 1), list(dfm['mouse2_color'].map({'black':0, 'white':1}).fillna(.5)))) # key: id * 4 + mouse_id
    m_c_di = m_c_di | dict(zip(list(dfm['id'] * 4 + 2), list(dfm['mouse3_color'].map({'black':0, 'white':1}).fillna(.5)))) # key: id * 4 + mouse_id
    m_c_di = m_c_di | dict(zip(list(dfm['id'] * 4 + 3), list(dfm['mouse4_color'].map({'black':0, 'white':1}).fillna(.5)))) # key: id * 4 + mouse_id
    df3['color1'] = (df3['id'] * 4 + df3['mid'] // 10).map(m_c_di).astype('float32')
    df3['color2'] = (df3['id'] * 4 + df3['mid'] % 10).map(m_c_di).astype('float32')

    # add strain of both mice; missing = .5. So values are 0/1/.5 - pretty much binary, leave it 'as is'
    # first, create mapping of id + mouse_id to strain
    m_s_di = dict(zip(list(dfm['id'] * 4 + 0), list(dfm['mouse1_strain'].map({'C57Bl/6J':0, 'BALB/c':1}).fillna(.5)))) # key: id * 4 + mouse_id
    m_s_di = m_s_di | dict(zip(list(dfm['id'] * 4 + 1), list(dfm['mouse2_strain'].map({'C57Bl/6J':0, 'BALB/c':1}).fillna(.5)))) # key: id * 4 + mouse_id
    m_s_di = m_s_di | dict(zip(list(dfm['id'] * 4 + 2), list(dfm['mouse3_strain'].map({'C57Bl/6J':0, 'BALB/c':1}).fillna(.5)))) # key: id * 4 + mouse_id
    m_s_di = m_s_di | dict(zip(list(dfm['id'] * 4 + 3), list(dfm['mouse4_strain'].map({'C57Bl/6J':0, 'BALB/c':1}).fillna(.5)))) # key: id * 4 + mouse_id
    df3['strain1'] = (df3['id'] * 4 + df3['mid'] // 10).map(m_s_di).astype('float32')
    df3['strain2'] = (df3['id'] * 4 + df3['mid'] % 10).map(m_s_di).astype('float32')

    # tracking method - no missing values in train. Split in 2 binaries
    tmp_di = dict(zip(list(dfm['id']), list(dfm['tracking_method'].map({'MARS':0, 'DeepLabCut':1, 'SLEAP':2}).fillna(0))))
    df3['tracking_method'] = df3['id'].map(tmp_di).fillna(0).astype('int8')
    df3['tracking_DeepLabCut'] = (df3['tracking_method'] == 1).astype('int8')
    df3['tracking_SLEAP'] = (df3['tracking_method'] == 2).astype('int8')
    df3.drop('tracking_method', axis=1, inplace=True)

    # arena shape - no missing values in train. Split in 3 binaries
    tmp_di = dict(zip(list(dfm['id']), list(dfm['arena_shape'].map({'rectangular':0, 'circular':1, 'split rectangluar':2, 'square':3}).fillna(0))))
    df3['arena_shape'] = df3['id'].map(tmp_di).fillna(0).astype('int8')
    df3['arena_shape_circular'] = (df3['arena_shape'] == 1).astype('int8')
    df3['arena_shape_split_rectangluar'] = (df3['arena_shape'] == 2).astype('int8')
    df3['arena_shape_square'] = (df3['arena_shape'] == 3).astype('int8')
    df3.drop('arena_shape', axis=1, inplace=True)
    
    # arena type - missing values. Split in 3 binaries
    tmp_di = dict(zip(list(dfm['id']), list(dfm['arena_type'].map({'resident-intruder':0, 'neutral':1, 'divided territories':2}).fillna(3)))) # others = 3
    df3['arena_type'] = df3['id'].map(tmp_di).fillna(0).astype('int8')
    df3['arena_type_neutral'] = (df3['arena_type'] == 1).astype('int8')
    df3['arena_type_divided'] = (df3['arena_type'] == 2).astype('int8')
    df3['arena_type_other'] = (df3['arena_type'] == 3).astype('int8')
    df3.drop('arena_type', axis=1, inplace=True)
    
    # frame rate: 13 values. Mostly correlates with lab, so let it be non-cat.
    tmp_di = dict(zip(list(dfm['id']), list(dfm['frames_per_second'].fillna(30))))
    df3['frames_per_second'] = df3['id'].map(tmp_di).fillna(30).astype('float32')

    # duration, log, rounded
    tmp_di = dict(zip(list(dfm['id']), list(dfm['video_duration_sec'].fillna(900))))
    df3['video_duration_sec'] = np.round(np.log(1 + df3['id'].map(tmp_di).fillna(6.8)), 1).astype('float32')

    # arena w/h
    tmp_di = dict(zip(list(dfm['id']), list(dfm['arena_width_cm'].fillna(35))))
    df3['arena_width_cm'] = df3['id'].map(tmp_di).fillna(35).astype('float32')
    tmp_di = dict(zip(list(dfm['id']), list(dfm['arena_height_cm'].fillna(30))))
    df3['arena_height_cm'] = df3['id'].map(tmp_di).fillna(30).astype('float32')

    # add number of possible actions. Log.
    df3['num_act'] = np.log(np.maximum(.5, mask.sum(-1) - 1)).astype(np.float32) # 1-11. Mostly 1.

    # bring in the frame rate
    fr_di = dict(zip(list(dfm['id']), list(dfm['frames_per_second'])))
    fr = np.round(df3['id'].map(fr_di).values, 0).astype(np.float32) # mostly 30, some are 25, 120 and 10
    fd = df3['video_frame'].values
    fd = (fd[1:] - fd[:-1]) * 30 / fr[1:]
    fd = np.maximum(0.25, fd).astype(np.float32) # to avoid /0; use .25 as a floor - for fr=120
    # feature: number of frames between steps
    df3['fd'] = 0
    df3['fd'].iloc[1:] = fd # apply qt to this
    df3['fd'] = df3['fd'].astype('float32')
    del fr

    # features for single action: 'tail_base':a,'ear_right':b,'ear_left':c,'nose':d,'body_center':e
    # dist mid-ear to tail base of mouse 1
    df3['dist_ear_tail_11'] = np.clip(np.sqrt((df3['x_1_b']/2 + df3['x_1_c']/2 - df3['x_1_a'])**2 + (df3['y_1_b']/2 + df3['y_1_c']/2 - df3['y_1_a'])**2).fillna(120).astype('float32').values, 5, 1000)
    # dist mid-ear to tail base
    df3['dist_ear_tail_12'] = np.clip(np.sqrt((df3['x_1_b']/2 + df3['x_1_c']/2 - df3['x_2_a'])**2 + (df3['y_1_b']/2 + df3['y_1_c']/2 - df3['y_2_a'])**2).fillna(230).astype('float32').values, 5, 3000)
    df3['dist_ear_tail_21'] = np.clip(np.sqrt((df3['x_2_b']/2 + df3['x_2_c']/2 - df3['x_1_a'])**2 + (df3['y_2_b']/2 + df3['y_2_c']/2 - df3['y_1_a'])**2).fillna(230).astype('float32').values, 5, 3000)
    # dist mid-ear to tail base [shifted by 10] of mouse 1
    df3['dist_ear_tail_s10_11'] = np.clip(np.sqrt((df3['x_1_b']/2 + df3['x_1_c']/2 - df3['x_1_a'].shift(10))**2 + (df3['y_1_b']/2 + df3['y_1_c']/2 - df3['y_1_a'].shift(10))**2).fillna(130).astype('float32').values, 5, 1000)
    # dist mid-ear to tail base [shifted by 10]
    df3['dist_ear_tail_s10_12'] = np.clip(np.sqrt((df3['x_1_b']/2 + df3['x_1_c']/2 - df3['x_2_a'].shift(10))**2 + (df3['y_1_b']/2 + df3['y_1_c']/2 - df3['y_2_a'].shift(10))**2).fillna(240).astype('float32').values, 5, 3000)
    df3['dist_ear_tail_s10_21'] = np.clip(np.sqrt((df3['x_2_b']/2 + df3['x_2_c']/2 - df3['x_1_a'].shift(10))**2 + (df3['y_2_b']/2 + df3['y_2_c']/2 - df3['y_1_a'].shift(10))**2).fillna(240).astype('float32').values, 5, 3000)

    # average mouse size - for feature scaling only
    size1 = np.maximum(.1, (df3['id'] * 4 + df3['mid'] // 10).map(m_size_di).values).astype(np.float32) # m_size_di; key: id * 4 + mouse_id
    size2 = np.maximum(.1, (df3['id'] * 4 + df3['mid'] % 10).map(m_size_di).values).astype(np.float32)
  
    # orientation of each mouse: best(based on vector length vs size) of ad or bc-perp
    # 1
    # ad
    ox = (df3['x_1_d'] - df3['x_1_a']).values / 3.2 # orientation vector; scale to the same mean length
    oy = (df3['y_1_d'] - df3['y_1_a']).values / 3.2 # orientation vector
    # bc perp
    oyt = -(df3['x_1_b'] - df3['x_1_c']).values # orientation vector
    oxt = (df3['y_1_b'] - df3['y_1_c']).values # orientation vector
    # blend
    osize1 = np.sqrt(ox**2 + oy**2)
    osizet = np.sqrt(oxt**2 + oyt**2)
    
    w = (np.abs(osize1 - size1 / 2.4) < np.abs(osizet - size1 / 2.4)) * 1 # pick the better one
    # w/wt
    ox = (ox * w + oxt * (1 - w)).astype(np.float32)
    oy = (oy * w + oyt * (1 - w)).astype(np.float32)

    # 2
    # ad
    ox2 = (df3['x_2_d'] - df3['x_2_a']).values / 3.2 # orientation vector; scale to the same mean length
    oy2 = (df3['y_2_d'] - df3['y_2_a']).values / 3.2 # orientation vector
    # bc perp
    oyt = -(df3['x_2_b'] - df3['x_2_c']).values # orientation vector
    oxt = (df3['y_2_b'] - df3['y_2_c']).values # orientation vector
    # blend
    osize2 = np.sqrt(ox2**2 + oy2**2)
    osizet = np.sqrt(oxt**2 + oyt**2)
    
    w = (np.abs(osize2 - size2 / 2.4) < np.abs(osizet - size2 / 2.4)) * 1 # pick the better one
    # w/wt
    ox2 = (ox2 * w + oxt * (1 - w)).astype(np.float32)
    oy2 = (oy2 * w + oyt * (1 - w)).astype(np.float32)
    del oxt, oyt, osizet, w, size1, size2
    gc.collect()

    osize1 = np.maximum(.1, np.sqrt(ox**2 + oy**2)).astype(np.float32) # length of orientation vector - use for all angles
    osize2 = np.maximum(.1, np.sqrt(ox2**2 + oy2**2)).astype(np.float32) # length of orientation vector - use for all angles

    # 'size' of each mouse
    a, b = fit_ellipse(df3, cols=['x_1_a','x_1_b','x_1_c','x_1_d','x_1_e','y_1_a','y_1_b','y_1_c','y_1_d','y_1_e']) # m1 axis
    df3['m1_maj'] = a
    df3['m1_min'] = b
    df3['m1_r'] = np.maximum(.5, a / b).astype(np.float32)
    a, b = fit_ellipse(df3, cols=['x_2_a','x_2_b','x_2_c','x_2_d','x_2_e','y_2_a','y_2_b','y_2_c','y_2_d','y_2_e']) # m2 axis
    df3['m2_maj'] = a
    df3['m2_min'] = b
    df3['m2_r'] = np.maximum(.5, a / b).astype(np.float32)
    del a, b
    gc.collect()


    # cos/sin of angle between mice
    df3['cos12'] = np.round(((ox * ox2 + oy * oy2) / osize1 / osize2), 4).astype(np.float32) # orientation ************************
    df3['sin12'] = np.round(((ox * oy2 - oy * ox2) / osize1 / osize2), 4).astype(np.float32) # orientation ************************


    # distance between noses
    df3['dist_nn_abs'] = np.maximum(2, np.sqrt((df3['x_1_d'] - df3['x_2_d'])**2 + (df3['y_1_d'] - df3['y_2_d'])**2)) # floor to make single=0 stand out less
    # distance nose to tail
    df3['dist_nt_abs'] = np.sqrt((df3['x_1_d'] - df3['x_2_a'])**2 + (df3['y_1_d'] - df3['y_2_a'])**2)
    # distance tail to nose
    df3['dist_tn_abs'] = np.sqrt((df3['x_2_d'] - df3['x_1_a'])**2 + (df3['y_2_d'] - df3['y_1_a'])**2)
    
                                                    
    # calculate absolute speed based on 3 best points
    w = np.ones([df3.shape[0]-1, 4], dtype=np.int32)
    sx1 = df3[['x_1_a', 'x_1_b', 'x_1_c', 'x_1_d']].values[1:] - df3[['x_1_a', 'x_1_b', 'x_1_c', 'x_1_d']].values[:-1]
    idx = np.abs(sx1 - sx1.mean(-1).reshape(-1,1)).argmax(-1)
    w[np.arange(w.shape[0]),idx] = 0
    sx1 = ((sx1 * w).sum(1) / w.sum(1)).astype(np.float32)

    w = np.ones([df3.shape[0]-1, 4], dtype=np.int32)
    sy1 = df3[['y_1_a', 'y_1_b', 'y_1_c', 'y_1_d']].values[1:] - df3[['y_1_a', 'y_1_b', 'y_1_c', 'y_1_d']].values[:-1]
    idx = np.abs(sy1 - sy1.mean(-1).reshape(-1,1)).argmax(-1)
    w[np.arange(w.shape[0]),idx] = 0
    sy1 = ((sy1 * w).sum(1) / w.sum(1)).astype(np.float32)

    w = np.ones([df3.shape[0]-1, 4], dtype=np.int32)
    sx2 = df3[['x_2_a', 'x_2_b', 'x_2_c', 'x_2_d']].values[1:] - df3[['x_2_a', 'x_2_b', 'x_2_c', 'x_2_d']].values[:-1]
    idx = np.abs(sx2 - sx2.mean(-1).reshape(-1,1)).argmax(-1)
    w[np.arange(w.shape[0]),idx] = 0
    sx2 = ((sx2 * w).sum(1) / w.sum(1)).astype(np.float32)

    w = np.ones([df3.shape[0]-1, 4], dtype=np.int32)
    sy2 = df3[['y_2_a', 'y_2_b', 'y_2_c', 'y_2_d']].values[1:] - df3[['y_2_a', 'y_2_b', 'y_2_c', 'y_2_d']].values[:-1]
    idx = np.abs(sy2 - sy2.mean(-1).reshape(-1,1)).argmax(-1)
    w[np.arange(w.shape[0]),idx] = 0
    sy2 = ((sy2 * w).sum(1) / w.sum(1)).astype(np.float32)
    del w, idx
    gc.collect()


    # distance nose to real tail and vv ********************************************
    df3['dist_nrt'] = np.sqrt((df3['x_1_d'] - df3['x_2_f'])**2 + (df3['y_1_d'] - df3['y_2_f'])**2)
    df3['dist_rtn'] = np.sqrt((df3['x_2_d'] - df3['x_1_f'])**2 + (df3['y_2_d'] - df3['y_1_f'])**2)
    # dist nt of mouse 1 only
    df3['dist_nrt_1'] = np.sqrt((df3['x_1_d'] - df3['x_1_f'])**2 + (df3['y_1_d'] - df3['y_1_f'])**2)
    idx = df3['x_2_f'] < -.99 # missing tail - set dist to xxx
    df3['dist_nrt'].loc[idx] = 150 # replace missing with mean values
    df3['dist_rtn'].loc[idx] = 150 # replace missing with mean values
    df3['dist_nrt_1'].loc[idx] = 100 # replace missing with mean values
    

    # distance between centers, absolute
    x1 = df3['x_1_e'].values # center: mean of all points
    df3.drop(['x_1_a', 'x_1_b', 'x_1_c', 'x_1_d', 'x_1_f'], axis=1, inplace=True)
    gc.collect()
    y1 = df3['y_1_e'].values # center
    df3.drop(['y_1_a', 'y_1_b', 'y_1_c', 'y_1_d', 'y_1_f'], axis=1, inplace=True)
    gc.collect()
    x2 = df3['x_2_e'].values # center
    df3.drop(['x_2_a', 'x_2_b', 'x_2_c', 'x_2_d', 'x_2_f'], axis=1, inplace=True)
    gc.collect()
    y2 = df3['y_2_e'].values # center
    df3.drop(['y_2_a', 'y_2_b', 'y_2_c', 'y_2_d', 'y_2_f'], axis=1, inplace=True)
    gc.collect()
    print('    dropped all x/y', int(time() - t0), 'sec')
    df3['dist_abs'] = np.maximum(2, np.sqrt((x1 - x2)**2 + (y1 - y2)**2)) # floor to make single=0 stand out less

   
    # features: angle of each mouse to vector between them
    dx = x1 - x2
    dy = y1 - y2
    df3['cos1_dist'] = ((ox * dx + oy * dy) / osize1 / np.maximum(.1, np.sqrt(dx**2 + dy**2))).astype(np.float32)  # orientation ************************
    df3['cos2_dist'] = ((ox2 * dx + oy2 * dy) / osize2 / np.maximum(.1, np.sqrt(dx**2 + dy**2))).astype(np.float32)  # orientation ************************
    del dx, dy, x1, x2, y1, y2
    gc.collect()

    # speed, absolute
    df3['speed1'] = 0
    df3['speed1'].iloc[1:] = np.sqrt(sx1**2 + sy1**2) / fd
    df3['speed1'] = df3['speed1'].astype('float32')
    
    df3['speed2'] = 0
    df3['speed2'].iloc[1:] = np.sqrt(sx2**2 + sy2**2) / fd
    df3['speed2'] = df3['speed2'].astype('float32')
    
    # angle between speed and orientation - to catch 'rear' action
    df3['cos_o_speed_1'] = 1
    df3['cos_o_speed_1'].iloc[1:] = (sx1 * ox[1:] + sy1 * oy[1:]) / np.sqrt(sx1**2 + sy1**2) / osize1[1:] # orientation ************************
    df3['cos_o_speed_1'] = df3['cos_o_speed_1'].fillna(1).astype('float32')
    df3['cos_o_speed_2'] = 1
    df3['cos_o_speed_2'].iloc[1:] = (sx2 * ox2[1:] + sy2 * oy2[1:]) / np.sqrt(sx2**2 + sy2**2) / osize2[1:] # orientation ************************
    df3['cos_o_speed_2'] = df3['cos_o_speed_2'].fillna(1).astype('float32')
    del osize1, osize2
    gc.collect()

    # cos s1 s2
    df3['cos_s1_s2'] = 1
    df3['cos_s1_s2'].iloc[1:] = (sx2 * sx1 + sy2 * sy1) / np.sqrt(sx2**2 + sy2**2) / np.sqrt(sx1**2 + sy1**2)
    df3['cos_s1_s2'] = np.round(df3['cos_s1_s2'].fillna(1), 4).astype('float32')
    del ox, oy, ox2, oy2
    gc.collect()

    # relative speed, absolute
    df3['rel_speed'] = 0
    df3['rel_speed'].iloc[1:] = np.sqrt((sx1 - sx2)**2 + (sy1 - sy2)**2) / fd
    df3['rel_speed'] = np.maximum(.005, df3['rel_speed']).astype('float32') # floor to make single=0 stand out less
    del fd, sx1, sx2, sy1, sy2
    gc.collect()
    
    # 'rear' - min cos b/w orientation and speed
    for dd in [3, 5, 10, 20]:
        for c in ['cos_o_speed_1']:
            df3[c+'_min_'+str(dd)] = df3[c].rolling(dd, center=True, min_periods=1).min().fillna(0).astype('float32')

    # speed: break between segments
    idx = (df3['id'] * 4 + df3['mid']).values # unique index of a series of frames
    idx = (idx[1:] != idx[:-1]).astype(np.int8)
    idx = np.concatenate((np.ones(1, dtype=np.int8), idx)) # now this is index of first frame of new series - use that for speed reset
    for c in ['speed1_abs', 'speed2_abs', 'speed1_sca', 'speed2_sca', 'rel_speed_abs', 'rel_speed_sca']:
        if c in df3.columns:
            df3[c].loc[idx] = 0

    # combine x12 and y12
    df3['x_e'] = (df3['x_1_e'] + df3['x_2_e']) / 2
    df3['y_e'] = (df3['y_1_e'] + df3['y_2_e']) / 2
    df3.drop(['x_1_e', 'x_2_e', 'y_1_e', 'y_2_e'], axis=1, inplace=True)
    gc.collect()

    if df3.isna().sum().sum() > 0: # check for NA
        print(df3.isna().sum())
        df3 = df3.fillna(method='ffill') # better than fill with 0
        df3 = df3.fillna(method='bfill') # JIC
    return df3





# x
N = 256 # length of data series ********************
def construct_x(df3, mask, train=0):
    global norm_di
    cols = list(df3.columns[3:]) # skip index(3)
    x = df3[cols].values.astype(np.float32)
    normo = cols.index('frames_per_second') # norm offset - skip binaries, start at 'frames_per_second'. 21. So norm 5 vars only
    qto = cols.index('fd') # qt offset - start at true* float, 'fd'. 26
    df3 = df3[df3.columns[:3]] # reduce size
    gc.collect()

    # skip QT. And skip binaries.
    for i in range(normo, qto): # skip binaries, skip qt
        if train:
            x_m = x[:,i].mean()
            x_s = x[:,i].std()
            norm_di['x_m'+str(i)] = x_m
            norm_di['x_s'+str(i)] = x_s
        else:
            x_m = norm_di['x_m'+str(i)]
            x_s = norm_di['x_s'+str(i)]
        x[:,i] -= x_m
        x[:,i] /= np.maximum(.00001, x_s)

    # QT x/y into uniform[0,1]/normal: first 'qto' vars have low cardinality
    # sometimes QT does not work - single actions have 0 for rel_speed, dist, ...
    if nn_mode == 'train':
        qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal', subsample=None, random_state=0, copy=False)
        qt = qt.fit(x[x[:,0]<.5, qto:]) # fit on data excluding single (single is the first var)
        joblib.dump(qt, model_dir + 'qt.pkl')
    else:
        qt = joblib.load(model_dir + 'qt.pkl')
    x[:,qto:] = qt.transform(x[:,qto:])

    x = x.astype(np.float32) # JIC
    l = np.minimum(N_LAB - 1, df3['id'].map(id_di).str[2].astype('int8').values)
   
    # trim/reshape everything
    df3 = df3.iloc[:N * (df3.shape[0]//N)] # trim df3
    x = x[:N * (x.shape[0]//N), :].reshape(-1, N, x.shape[-1])
    mask = mask[:N * (mask.shape[0]//N), :].reshape(-1, N, mask.shape[-1])
    l = l[:df3.shape[0]].reshape(-1, N)
    return x, df3, mask, l





# real submission
def prepare_sub(yp, df3, mp, mp3, sm=0, sm1=0):
    # calc yp2
    lab = df3['id'].map(id_di).str[2].values[:yp.shape[0]].astype(np.int8)
    if sm1 == 0:
        yp2 = ((yp > mp[lab, :]) * 1).astype(np.int8)
    else:
        yp2 = ((savgol_filter(yp, window_length=sm1, polyorder=1, axis=0) > mp[lab, :]) * 1).astype(np.int8)

    # round of smoothing, to eliminate runs of 1: reduces size of sub, does not really improve the score.
    if sm > 1:
        yp2 = np.round(savgol_filter(yp2, window_length=sm, polyorder=1, axis=0), 0).astype(np.int8) # 10/1 seems best

    # reset prediction on id+mid change - need this to avoid submission errors!
    d2 = df3[['id', 'video_frame', 'mid', 'video_frame']].values
    d2[:,3] = d2[:,0] * (d2[:,2].max() + 1) + d2[:,2] # combine id and mid
    d2 = np.concatenate((d2[-1:,:], d2), axis=0) # make it longer - append 1 to the front; this shifts it forward by 1!!!!!!
    idx = d2[1:yp2.shape[0]+1,3] != d2[:yp2.shape[0],3] # this is the index of the first frame in a series
    yp2[idx,:] = 0
    
    # remove duplicate actions
    idx = yp2.sum(-1) > 1
    if idx.sum() > 0:
        idx2 = (yp[idx,:] / mp3[lab[idx], :]).argmax(-1) # highest adjusted probability
        yp2[idx,:] *= 0 # for now, just remove all of them
        yp2[idx,idx2] = 1 # assign action to ...
    
    # loop over actions
    res = []
    for j in range(yp2.shape[1]):
        i1 = 0
        a = yp2[0,j] # init to starting status
        idx = yp2[1:,j] != yp2[:-1,j]
        ii = 1 + np.arange(yp2.shape[0] - 1)[idx] # these are indices of prediction change
        for i in ii:
            if a == 0: # start action
                a, i1 = 1, i
            else: # stop action and record it
                a = 0
                res.append([d2[i1+1,0], d2[i1+1,2], j, d2[i1+1,1], 1 + d2[i,1]]) # id, mid, action, start, stop; shift by 1 + lengthen by 1!!!
    sub = pd.DataFrame(res)
    del res, yp2
    gc.collect()
    sub.columns = ['id', 'mid', 'action', 'start_frame', 'stop_frame']
    sub['id'] = sub['id'].map(id_di)
    sub['video_id'] = sub['id'].str[1]
    sub['action'] = sub['action'].map(action_di2)
    sub['agent_id']  = sub['mid'].map({1:'mouse1', 2:'mouse1', 3:'mouse1', 12:'mouse2', 13:'mouse2', 23:'mouse3', 0:'mouse1', 10:'mouse2', 11:'mouse2', 20:'mouse3', 21:'mouse3', 22:'mouse3', 30:'mouse4', 31:'mouse4', 32:'mouse4', 33:'mouse4'})
    sub['target_id'] = sub['mid'].map({1:'mouse2', 2:'mouse3', 3:'mouse4', 12:'mouse3', 13:'mouse4', 23:'mouse4', 0:'mouse1', 10:'mouse1', 11:'mouse2', 20:'mouse1', 21:'mouse2', 22:'mouse3', 30:'mouse1', 31:'mouse2', 32:'mouse3', 33:'mouse4'})
    # relabel self-actions
    sub['target_id'].loc[sub['target_id'] == sub['agent_id']] = 'self'
    sub['row_id'] = sub.index
    sub = sub[['row_id','video_id','agent_id','target_id','action','start_frame','stop_frame']] # drop and reorder, JIC
    return sub


NN execution

In [3]:
# code *******************************************************************************************************************************************
if nn_mode == 'submit': # submit part *********************************************************************
    verbose = 1
    norm_di = joblib.load(model_dir + 'norm_di.pkl')
    
    df, dfm, id_di, id_di2, m_size_di = read_data(train = 0) # read test data
    df3 = prepare_pairs() # format into mouse pairs
    df3, mask = prepare_mask(df3) # construct 'behaviors_labeled' mask
    df3 = FE(df3) # FE
    x, df3, mask, l0 = construct_x(df3, mask, train=0) # x

    # get the test prediction
    S = 1024 # batch size
    yp = np.zeros([x.shape[0], x.shape[1], N_ACT], dtype=np.float32) # predicted - skip 'no action'
    run_mult = {3:0.15, 4:0.08, 9:0.03, 16:0.17} # blend 4 NN models with these weights
    with tf.device('/GPU:0'):
        for run in [3,4,9,16]:
          for fold in range(folds):
            model = tf.keras.models.load_model(model_dir+'model'+str(fold)+'_'+str(run)+'.keras')
            for i in range(1 + x.shape[0]//S):
                i1 = S * i
                i2 = np.minimum(S * (i+1), x.shape[0])
                if run == 16: # N=512, use special logic
                    if (i2 - i1)%2 == 1: # make it even
                        i2 -= 1
                    yp[i1:i2,:,:] = (yp[i1:i2,:,:].ravel() + run_mult[run] * model.predict((x[i1:i2,:,:].reshape(-1,N*2,x.shape[-1]), mask[i1:i2,:,:].reshape(-1,N*2,mask.shape[-1]), l0[i1:i2,:].reshape(-1,N*2)), verbose=0, batch_size=S)[:,:,1:].ravel() / folds).reshape(-1,N,yp.shape[-1])
                else:
                    yp[i1:i2,:,:] += run_mult[run] * model.predict((x[i1:i2,:,:], mask[i1:i2,:,:], l0[i1:i2,:]), verbose=0, batch_size=S)[:,:,1:] / folds # skip 'no action'

    # TTA - shift by N/2
    x = x.reshape(-1, x.shape[-1])[N//2:-N//2].reshape(-1, N, x.shape[-1])
    mask = mask.reshape(-1, mask.shape[-1])[N//2:-N//2].reshape(-1, N, mask.shape[-1])
    l0 = l0.ravel()[N//2:-N//2].reshape(-1, N)
    yp2 = np.zeros([x.shape[0], x.shape[1], N_ACT], dtype=np.float32) # predicted - skip 'no action'
    with tf.device('/GPU:0'):
        for run in [3,4,9,16]:
          for fold in range(folds):
            model = tf.keras.models.load_model(model_dir+'model'+str(fold)+'_'+str(run)+'.keras')
            for i in range(1 + x.shape[0]//S):
                i1 = S * i
                i2 = np.minimum(S * (i+1), x.shape[0])
                if run == 16: # N=512, use special logic
                    if (i2 - i1)%2 == 1: # make it even
                        i2 -= 1
                    yp2[i1:i2,:,:] = (yp2[i1:i2,:,:].ravel() + run_mult[run] * model.predict((x[i1:i2,:,:].reshape(-1,N*2,x.shape[-1]), mask[i1:i2,:,:].reshape(-1,N*2,mask.shape[-1]), l0[i1:i2,:].reshape(-1,N*2)), verbose=0, batch_size=S)[:,:,1:].ravel() / folds).reshape(-1,N,yp.shape[-1])
                else:
                    yp2[i1:i2,:,:] += run_mult[run] * model.predict((x[i1:i2,:,:], mask[i1:i2,:,:], l0[i1:i2,:]), verbose=0, batch_size=S)[:,:,1:] / folds # skip 'no action'

    # reduce memory in submission; still need df3
    del x, mask, l0
    gc.collect()
    
    # unroll
    yp = yp.reshape(-1, N_ACT)
    yp2 = yp2.reshape(-1, N_ACT)
    yp[N//2:-N//2,:] = (yp[N//2:-N//2,:] + yp2) / 2


    yp = yp / 0.43 # undo blending weights ********************************************************************************

    # reduce memory in submission; still need df3
    del yp2
    gc.collect()

    # do not submit anything here, blend predictions in the end *******************************************************
    print('done', int(time() - t0), 'sec')





if nn_mode != 'submit': # train/validate part *****************************************************************************************************
    df, dfm, dfa, dfa0, id_di, id_di2, m_size_di = read_data(train = 1) # read train data and combine all body parts
    print('read', df.shape, dfa.shape, dfm.shape, int(time() - t0), 'sec')

    df3 = prepare_pairs() # format into mouse pairs
    print('format into mouse pairs', df3.shape, int(time() - t0), 'sec') # exacly double the individual count.
    
    df3, mask = prepare_mask(df3) # construct 'behaviors_labeled' mask.
    print('masks', mask.shape, int(time() - t0), 'sec') # drop over 1/2 unlabeled ones (mostly self-actions)

    df3 = FE(df3) # FE
    print('FE', df3.shape, int(time() - t0), 'sec')
    print(df3.columns)
    print(df3.dtypes.value_counts())

    # target: add it as the last column in the data
    dfa['action'] = dfa['action'].map(action_di).astype('int8')
    dfa['mid'] = dfa['target_id'] + dfa['agent_id'] * 10 # 1, 2, 3, 12, 13, 23, etc - same as df3
    dfa.drop(['agent_id', 'target_id'], axis=1, inplace=True)
    dfa = dfa.sort_values(by=['id', 'mid', 'start_frame']).reset_index(drop=True)

    # expand all frames
    res = np.zeros([(dfa['stop_frame']-dfa['start_frame']).sum(), 4], dtype=np.int32) # 5 Mil
    i1 = 0
    for i in range(dfa.shape[0]):
        f1 = dfa['start_frame'].iloc[i]
        f2 = dfa['stop_frame'].iloc[i]
        i2 = i1 + f2 - f1
        res[i1:i2,0] = dfa['id'].iloc[i]
        res[i1:i2,1] = np.arange(f1, f2)
        res[i1:i2,2] = dfa['mid'].iloc[i]
        res[i1:i2,3] = dfa['action'].iloc[i]
        i1 = i2
    dfa2 = pd.DataFrame(res)
    dfa2.columns =['id', 'video_frame', 'mid', 'action']

    # merge with data
    df3 = df3.merge(dfa2, on=['id', 'video_frame', 'mid'], how='left')
    df3['action'] = df3['action'].fillna(-1).astype('int8')
    del dfa2, res
    gc.collect()
    print('target', df3.shape, 'num records with action', (df3['action']>=0).sum(), int(time() - t0), 'sec')

    # model: prep data
    # y, never shaped into N
    y0 = 1 + df3['action'].values # 0 to N_ACT (N_ACT+1 total, incl 'no action', which is 0)
    y0 = y0[:N * (y0.shape[0]//N)] # trim
    df3.drop('action', axis=1, inplace=True)
    
    x, df3, mask, l0 = construct_x(df3, mask, train=1) # x
    if nn_mode == 'train': # save scaling data in train only
        joblib.dump(norm_di, model_dir + 'norm_di.pkl')
    print('model: prep data', x.shape, int(time() - t0), 'sec')

   
    # model: training
    DR1         = 0.3
    LL2         = 1e-5
    filters     = 256
    BS          = 32
    D1          = 256
    act         = 'selu'
    lr0         = 5e-3
    lrd         = 25

    verbose = 1
    np.random.seed(seed)
    random.seed(seed)
    yp = np.zeros([x.shape[0], N, N_ACT], dtype=np.float32) # predicted - skip 'no action', so N_ACT and not N_ACT+1
    with tf.device('/GPU:0'):
        class data_gen(Sequence): # data generator
            def __init__(self, idx, batch_size, train): # constructor: save all data locally
                self.idx, self.batch_size, self.train = idx.astype(np.int32), batch_size, train
                super().__init__(max_queue_size=4000)
                return

            def __len__(self): # returns number of batches
                return math.ceil(len(self.idx) / self.batch_size)

            def __getitem__(self, b): # returns one batch
                idx2 = self.idx[self.batch_size * b:self.batch_size * (b + 1)]
                
                # train-time augmentation: shift all by random amount from 0 to 128
                if self.train:
                    idx2 = np.minimum(idx2, x.shape[0] - 2) # cap the index at ... - 1
                    o = np.random.randint(N)
                    xb = x.reshape(-1, x.shape[-1])[o:o-N,:].reshape(-1, N, x.shape[-1])[idx2,:,:]
                    mb = mask.reshape(-1, mask.shape[-1])[o:o-N,:].reshape(-1, N, mask.shape[-1])[idx2,:,:]
                    lb = l0.ravel()[o:o-N].reshape(-1, N)[idx2,:]
                    yb = y0[o:o-N].reshape(-1, N)[idx2,:]
                else: # do not augment validation data; best w/b to average several augmentations, but never to pick a random one!
                    xb, mb, lb, yb = x[idx2,:,:], mask[idx2,:,:], l0[idx2,:], y0.reshape(-1, N)[idx2,:]
                return ((xb, mb, lb), yb)

            def on_epoch_end(self): # this shuffle greatly improves convergence
                if self.train: # do not shuffle va!
                    np.random.shuffle(self.idx)

        kf = StratifiedGroupKFold(n_splits=folds, shuffle=False, random_state=None) # groups: videos
        l1 = np.median(df3['id'].map(id_di).str[2].values[:N * x.shape[0]].reshape(-1, N), axis=-1).astype(np.int16) # stratified by lab
        for fold, (tr_ind, va_ind) in enumerate(kf.split(x, l1, np.median(df3['id'].values[:N * x.shape[0]].reshape(-1, N), axis=-1).astype(np.int16))):
            if nn_mode == 'train': # train the model *****************************************************************************************
                i1 = Input(shape=(N, x.shape[2],), dtype='float32', name='x')
                i2 = Input(shape=(N, N_ACT + 1,), dtype='int8', name='mask')
                i3 = Input(shape=(N, ), dtype='int8', name='lab')

                i3b= Embedding(input_dim=N_LAB, output_dim=8, input_length=1, name='embed1')(i3)
                i1a= Concatenate(name='x_l1')([i1, i3b])

                l1 = Bidirectional(LSTM(filters//2, return_sequences=True, kernel_regularizer=L2(LL2), recurrent_regularizer=L2(LL2)), name='LSTM1')(sequences=i1a)
                l1 = Dropout(DR1, name='DR1')(l1)

                d1 = Dense(D1, activation=act, kernel_initializer='lecun_normal', name='D1')(l1)
                
                i3a= Embedding(input_dim=N_LAB, output_dim=4, input_length=1, name='embed2')(i3)
                do = Concatenate(name='x_l2')([d1, i3a])
                
                o  = Dense(N_ACT + 1, activation=None, name='Do')(do) # N_ACT+1 actions, including 'no action'
                
                o  = o * i2 - (1 - i2) * 6.0 # apply mask and set excluded actions to -6 (1/400)
                o  = Softmax(name='o')(o)
                
                model = tf.keras.Model(inputs=(i1, i2, i3), outputs=o)
                if verbose and fold == 0: print(model.summary())
                sc = CosineDecayRestarts(initial_learning_rate=lr0*0.6, first_decay_steps=4) # reduce lr0
                es = EarlyStopping(monitor='val_loss', start_from_epoch=5, patience=69, mode='min', verbose=1*verbose, restore_best_weights=True)
                
                class print_lr(Callback):
                    def on_epoch_end(self, epoch, logs=None):
                        print('epoch', epoch + 1, 'lr', np.round(np.array(self.model.optimizer.learning_rate), 6), ' ', end='')
                
                model.compile(optimizer=Adam(learning_rate=sc), loss=SparseCategoricalCrossentropy())
                tr = data_gen(tr_ind, BS, 1)
                va = data_gen(va_ind, BS, 0)
                h = model.fit(x=tr, validation_data=va, epochs=74, callbacks=[es, print_lr()], verbose=2*verbose)
                model.save(model_dir+'model'+str(fold)+'.keras')
                del tr, va
                gc.collect()
                
            if nn_mode == 'validate': # load saved model ****************************************************************************
                model = tf.keras.models.load_model(model_dir+'model'+str(fold)+'.keras')
                
            S = 1024 # prediction batch size
            for i in range(1 + len(va_ind)//S):
                i1, i2 = S * i, np.minimum(S * (i + 1), len(va_ind))
                idx = va_ind[i1:i2]
                yp[idx,:,:] = model.predict((x[idx,:,:], mask[idx,:,:], l0[idx,:]), verbose=0, batch_size=S)[:,:,1:] # skip 'no action'
            if verbose: print('    finished fold', fold, int(time() - t0), 'sec')
            del tr_ind, va_ind, model
            gc.collect()

    # reduce ram usage
    del x, mask
    gc.collect()
    
    # unroll
    yp = yp.reshape(-1, N_ACT)
    l0 = l0.ravel()
    
    # drop all labs past 15 - don't need them after training! **********************
    idx = l0 < 15
    y0 = y0[idx]
    yp = yp[idx,:]
    df3 = df3.loc[idx].reset_index(drop=True)
    dfa0 = dfa0.loc[(dfa0['lab_id'].str[:8] != 'CalMS21_') & (dfa0['lab_id'].str[:6] != 'CRIM13')]

    # score the prediction
    dft = pd.DataFrame(y0)
    dft.columns = ['y']
    dft['id'] = df3['id']
    dft['l'] = dft['id'].map(id_di).str[2].astype('int8')
    dft['video_frame'] = df3['video_frame']
    dft['w'] = df3['video_frame']
    dft['w'].iloc[1:] = np.maximum(1, df3['video_frame'].values[1:dft.shape[0]] - df3['video_frame'].values[:dft.shape[0]-1])

    # reset prediction on id+mid change
    d2 = df3[['id', 'video_frame', 'mid', 'video_frame']].values
    d2[:,3] = d2[:,0] * (d2[:,2].max() + 1) + d2[:,2] # combine id and mid
    d2 = np.concatenate((d2[-1:,:], d2), axis=0) # make it longer
    idx = d2[1:yp.shape[0]+1,3] != d2[:yp.shape[0],3] # this is the index of the first frame in a series
    yp[idx,:] = 0
    dft['w'].loc[idx] = 0
    del d2

    # find threshold to optimize F1. By action + lab.
    mp = np.ones([15, N_ACT], dtype=np.float32) # 15 labs by N_ACT actions
    li = []
    for l1 in range(1 + dft['l'].max()):
        idx = dft['l'] == l1
        dft1 = dft.loc[idx].reset_index(drop=True)
        yp1 = yp[idx,:]
        for i in range(N_ACT):
            if (dft1['y'] == 1 + i).sum() > 0:
                a = -yp1[:,i].ravel() # predicted probability
                b = (dft1['y'] == 1 + i).values.astype(np.int8) # y
                w = dft1['w'].values.astype(np.int8) # w = frames per observation
                b = b * w # y, with weight
                idx = np.argsort(a, order=None) # this is where all the runtime goes
                a = -a[idx]
                b = b[idx]
                w = w[idx]
                b = b.cumsum()
                w = w.cumsum() # predicted count, with weight
                f = 2 * b / (b[-1] + w) # here b[-1] id yc(constant), w is ypc(cumulative by threshold)
                j = f.argmax()
                mp[l1, i] = (a[j] + a[j-1]) / 2
                li.append([l1, f[j], i, b[j], b[-1], w[j]]) # save expected F1
    li2 = pd.DataFrame(li).groupby(0)[1].mean() # score by lab. lab/score/action/tp/yc/ypc
    if verbose: print('mp: best score:', np.round(li2.values.mean(), 3), int(time() - t0), 'sec')
    if nn_mode == 'train': # save mp
        joblib.dump(mp, model_dir + 'mp.pkl') # save this for training only!!!

    # construct yp2 and my score ********
    lab = dft['l'].values[:yp.shape[0]]
    yp2 = ((yp > mp[lab, :]) * 1).astype(np.int8)

    # remove duplicate actions
    idx = yp2.sum(-1) > 1
    if idx.sum() > 0:
        idx2 = (yp[idx,:] / mp[lab[idx],:]).argmax(-1) # highest adjusted probability
        yp2[idx,:] *= 0 # for now, just remove all of them
        yp2[idx,idx2] = 1 # assign action to ...

    # compute my version of score by lab; it does not match exactly because of missing frames
    dft['yp'] = (yp2 * np.array(np.arange(1, N_ACT+1)).reshape(1, -1)).sum(-1).astype('int8')
    dft['match'] = (dft['yp'] != 0 ) * (dft['y'] == dft['yp']) * 1
    dft['mid'] = df3['mid']
    dft['m2'] = dft['y'] * dft['match']

    # start with TP
    dft2 = dft.groupby(['m2', 'l'])['w'].sum().reset_index()
    dft2.columns = ['y','l','TP']

    # add FP(ypc)
    dft3 = dft.groupby(['yp','l'])['w'].sum().reset_index()
    dft3.columns = ['y', 'l','FP']
    dft2 = dft2.merge(dft3, on=['y', 'l'], how='outer')

    # add FN(yc)
    dft3 = dft.groupby(['y','l'])['w'].sum().reset_index()
    dft3.columns = ['y', 'l','FN']
    dft2 = dft2.merge(dft3, on=['y', 'l'], how='outer')
    
    # F1
    dft2['TP'] = dft2['TP'].fillna(0)
    dft2['FN'] = dft2['FN'].fillna(0) - dft2['TP']
    dft2['FP'] = dft2['FP'].fillna(0) - dft2['TP']
    dft2['F1'] = dft2['TP'] * 2 / (dft2['TP'] * 2 + dft2['FP'] + dft2['FN']) # only now calc F1
    dft2 = dft2.loc[dft2['y'] > 0].reset_index(drop=True) # drop 0
    dft3 = dft2.groupby('l')['F1'].mean().reset_index() # F1 by lab
    if verbose: print('my score by lab', np.round(dft3['F1'].mean(), 3), int(time() - t0), 'sec')

    # score
    sub = prepare_sub(yp, df3) # real submission
    sc = score(dfa0, sub, row_id_column_name='row_id')
    if verbose: print('score_predicted', np.round(sc, 3), int(time() - t0), 'sec***********')


     0 0 AdaptableSnail/438887472 73692
    finished reading files 1 sec
    interpolating missing values... 1 sec
    proportion without behaviors_labeled, drop 0.0
    dropped all x/y 5 sec


I0000 00:00:1765814678.365107      28 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1765814681.415288      86 cuda_dnn.cc:529] Loaded cuDNN version 90300


done 69 sec


# Second, do XGB prediction. Save result in "df", which combined metadata and the predictions

In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import numpy as np
import os, json, gc, glob, joblib, warnings, itertools
from scipy.signal import savgol_filter
from time import time
t0 = time()
warnings.filterwarnings('ignore')


folds = 5
verbose = 1
seed = 13

class CFG:
    train_path = "/kaggle/input/MABe-mouse-behavior-detection/train.csv"
    test_path = "/kaggle/input/MABe-mouse-behavior-detection/test.csv"
    train_annotation_path = "/kaggle/input/MABe-mouse-behavior-detection/train_annotation"
    train_tracking_path = "/kaggle/input/MABe-mouse-behavior-detection/train_tracking"
    test_tracking_path = "/kaggle/input/MABe-mouse-behavior-detection/test_tracking"

    model_name  = "/kaggle/input/xgb-data-v1/lgb"
    model_name2 = "/kaggle/input/xgb-data-v2/lgb"
    
    mode = 'submit' # 'validate'/'check'/'submit'

XGB function definitions (mostly)

In [5]:
## Data loading and preprocessing
train = pd.read_csv(CFG.train_path)
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~lab_id.str.startswith('MABe22_')")
test = pd.read_csv(CFG.test_path)
body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

if CFG.mode != 'submit':
    dfm = train.copy()
else:
    dfm = test.copy()



## Creating solution data
def create_solution_df(dataset):
    solution = []
    for _, row in dataset.iterrows():
    
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'): 
            continue
        
        video_id = row['video_id']
        path = f"{CFG.train_annotation_path}/{lab_id}/{video_id}.parquet"
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            continue
    
        annot['lab_id'] = lab_id
        annot['video_id'] = video_id
        annot['behaviors_labeled'] = row['behaviors_labeled']
        annot['target_id'] = np.where(annot.target_id != annot.agent_id, annot['target_id'].apply(lambda s: f"mouse{s}"), 'self')
        annot['agent_id'] = annot['agent_id'].apply(lambda s: f"mouse{s}")
        solution.append(annot)
    
    solution = pd.concat(solution)

    # drop excluded actions
    solution = solution.loc[solution['action'].map(lambda x: x not in ['ejaculate', 'dominancemount', 'genitalgroom', 'disengage'])]
    return solution

if CFG.mode != 'submit':
    solution = create_solution_df(train_without_mabe22)





## Data generator
drop_body_parts =  [
    'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
    'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint'
]


def generate_mouse_data(dataset, traintest, traintest_directory, generate_single=True, generate_pair=True):
    vid_count = 0
    for _, row in dataset.iterrows():
        lab_id = row.lab_id
        if lab_id.startswith('MABe22') or type(row.behaviors_labeled) != str: 
            continue
        if traintest == 'test' and lab_id in ['CRIM13', 'CalMS21_supplemental', 'CalMS21_task1', 'CalMS21_task2']: # skip excluded labs in test
            continue
        
        video_id = row.video_id
        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path) # this reads the data *************************************************************************************
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")

        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])

        del vid
        gc.collect()
        
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
        pvid /= row.pix_per_cm_approx
        
        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
        
        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                continue

        vid_count += 1
        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'")
            skip = 0
            if lab_id == 'AdaptableSnail' and row.frames_per_second == 25:
                skip = 1
                print('skipping bad singles') # print ****************************************************************
            if len(vid_behaviors_subset) > 0 and skip == 0: # skip bad singles
                for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                    try:
                        mouse_id = int(mouse_id_str[-1])
                        vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)
                        single_mouse = pvid.loc[:, mouse_id]
                        assert len(single_mouse) == len(pvid)
                        single_mouse_meta = pd.DataFrame({
                            'lab_id': lab_id,
                            'video_id': video_id,
                            'agent_id': mouse_id_str,
                            'target_id': 'self',
                            'video_frame': single_mouse.index
                        })
                        if traintest == 'train':
                            single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index)
                            annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                            for i in range(len(annot_subset)):
                                annot_row = annot_subset.iloc[i]
                                single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'] - 1, annot_row.action] = 1.0
                            yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                        else:
                            yield 'single', single_mouse, single_mouse_meta, vid_agent_actions
                    except KeyError:
                        pass

        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            skip = 0
            if lab_id == 'AdaptableSnail' and row.frames_per_second == 25:
                skip = 1
                print('skipping bad pairs') # print ****************************************************************
            if len(vid_behaviors_subset) > 0 and skip == 0: # skip bad pairs
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2):
                    agent_str = f"mouse{agent}"
                    target_str = f"mouse{target}"
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B'])
                    assert len(mouse_pair) == len(pvid)
                    mouse_pair_meta = pd.DataFrame({
                        'lab_id': lab_id,
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index
                    })
                    if traintest == 'train':
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'] - 1, annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions


## Transforming coordinates
def safe_rolling(series, window, func, min_periods=None):
    if min_periods is None:
        min_periods = max(1, window // 4)
    return series.rolling(window, min_periods=min_periods, center=True).apply(func, raw=True)

def _scale(n_frames_at_30fps, fps, ref=30.0):
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

def _fps_from_meta(meta_df, fallback_lookup, default_fps=30.0):
    if 'frames_per_second' in meta_df.columns and pd.notnull(meta_df['frames_per_second']).any():
        return float(meta_df['frames_per_second'].iloc[0])
    vid = meta_df['video_id'].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))

def add_curvature_features(X, center_x, center_y, fps):
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()

    cross_prod = vel_x * acc_y - vel_y * acc_x
    vel_mag = np.sqrt(vel_x**2 + vel_y**2)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)

    for w in [25, 50, 75]:
        ws = _scale(w, fps)
        X[f'curv_mean_{w}'] = curvature.rolling(ws, min_periods=max(1, ws // 5)).mean()

    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    w = 30
    ws = _scale(w, fps)
    X[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 5)).sum()

    return X

def add_multiscale_features(X, center_x, center_y, fps):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)

    scales = [20, 40, 60, 80]
    for scale in scales:
        ws = _scale(scale, fps)
        if len(speed) >= ws:
            X[f'sp_m{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).mean()
            X[f'sp_s{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).std()

    if len(scales) >= 2 and f'sp_m{scales[0]}' in X.columns and f'sp_m{scales[-1]}' in X.columns:
        X['sp_ratio'] = X[f'sp_m{scales[0]}'] / (X[f'sp_m{scales[-1]}'] + 1e-6)

    return X

def add_state_features(X, center_x, center_y, fps):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    w_ma = _scale(15, fps)
    speed_ma = speed.rolling(w_ma, min_periods=max(1, w_ma // 3)).mean()

    try:
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)

        for window in [20, 40, 60, 80]:
            ws = _scale(window, fps)
            if len(speed_states) >= ws:
                for state in [0, 1, 2, 3]:
                    X[f's{state}_{window}'] = (
                        (speed_states == state).astype(float)
                        .rolling(ws, min_periods=max(1, ws // 5)).mean()
                    )
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                X[f'trans_{window}'] = state_changes.rolling(ws, min_periods=max(1, ws // 5)).sum()
    except Exception:
        pass

    return X

def add_longrange_features(X, center_x, center_y, fps):
    for window in [30, 60, 120]:
        ws = _scale(window, fps)
        if len(center_x) >= ws:
            X[f'x_ml{window}'] = center_x.rolling(ws, min_periods=max(5, ws // 6)).mean()
            X[f'y_ml{window}'] = center_y.rolling(ws, min_periods=max(5, ws // 6)).mean()

    for span in [30, 60, 120]:
        s = _scale(span, fps)
        X[f'x_e{span}'] = center_x.ewm(span=s, min_periods=1).mean()
        X[f'y_e{span}'] = center_y.ewm(span=s, min_periods=1).mean()

    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)  # cm/s
    for window in [30, 60, 120]:
        ws = _scale(window, fps)
        if len(speed) >= ws:
            X[f'sp_pct{window}'] = speed.rolling(ws, min_periods=max(5, ws // 6)).rank(pct=True)

    return X

def add_interaction_features(X, mouse_pair, fps, bp):
    rel_x = mouse_pair['A'][bp]['x'] - mouse_pair['B'][bp]['x']
    rel_y = mouse_pair['A'][bp]['y'] - mouse_pair['B'][bp]['y']
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)

    A_vx = mouse_pair['A'][bp]['x'].diff()
    A_vy = mouse_pair['A'][bp]['y'].diff()
    B_vx = mouse_pair['B'][bp]['x'].diff()
    B_vy = mouse_pair['B'][bp]['y'].diff()

    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    for window in [30, 60]:
        ws = _scale(window, fps)
        X[f'A_ld{window}'] = A_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        X[f'B_ld{window}'] = B_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()

    approach = -rel_dist.diff()
    chase = approach * B_lead
    w = 30
    ws = _scale(w, fps)
    X[f'chase_{w}'] = chase.rolling(ws, min_periods=max(1, ws // 6)).mean()

    for window in [60, 120]:
        ws = _scale(window, fps)
        A_sp = np.sqrt(A_vx**2 + A_vy**2)
        B_sp = np.sqrt(B_vx**2 + B_vy**2)
        X[f'sp_cor{window}'] = A_sp.rolling(ws, min_periods=max(1, ws // 6)).corr(B_sp)

    return X


def roll_f(X, fps): # add rolled features
    for dd in [3, 5, 10, 20]:
        for c in ['1speed1', 'speed1', 'speed2', '12+nose+tail_base', '12+nose+body_center', '12+nose+nose', '12+neck+neck', '12+body_center+nose', 'lateral_left+lateral_right', 'ear_left+ear_right']:
            if c in X.columns:
                win = int(dd * fps / 30 + 0.5) # rolling window, adjusted for fps
                X['roll_'+c+'_mi_'+str(dd)] = X[c].rolling(win, center=True, min_periods=1).min().astype('float32')
                X['roll_'+c+'_ma_'+str(dd)] = X[c].rolling(win, center=True, min_periods=1).max().astype('float32')
                X['roll_'+c+'_me_'+str(dd)] = X[c].rolling(win, center=True, min_periods=1).mean().astype('float32')
    return X





def transform_single(single_mouse, body_parts_tracked, fps, section, meta): #************************************************************************
    available_body_parts = single_mouse.columns.get_level_values(0)
    
    X = pd.DataFrame({
        f"{p1}+{p2}": np.square(single_mouse[p1] - single_mouse[p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.combinations(body_parts_tracked, 2)
        if p1 in available_body_parts and p2 in available_body_parts
    })
    X = X.reindex(columns=[f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        lag = _scale(10, fps)
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)
        speeds = pd.DataFrame({
            'sp_lf': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
            'sp_rt': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
            'sp_lf2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
            'sp_rt2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'cx_m{w}'] = cx.rolling(ws, **roll).mean()
            X[f'cy_m{w}'] = cy.rolling(ws, **roll).mean()
            X[f'cx_s{w}'] = cx.rolling(ws, **roll).std()
            X[f'cy_s{w}'] = cy.rolling(ws, **roll).std()
            X[f'x_rng{w}'] = cx.rolling(ws, **roll).max() - cx.rolling(ws, **roll).min()
            X[f'y_rng{w}'] = cy.rolling(ws, **roll).max() - cy.rolling(ws, **roll).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).sum()**2 +
                                     cy.diff().rolling(ws, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).var() +
                                   cy.diff().rolling(ws, min_periods=1).var())

        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)

    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt_dist.shift(l)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l)

    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                        (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        for off in [-30, -20, -10, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)
        w = _scale(30, fps)
        X['ear_con'] = ear_d.rolling(w, min_periods=1, center=True).std() / \
                       (ear_d.rolling(w, min_periods=1, center=True).mean() + 1e-6)


    # angle between speed and orientation
    sx1 = single_mouse['tail_base']['x'].values[1:] - single_mouse['tail_base']['x'].values[:-1]
    sy1 = single_mouse['tail_base']['y'].values[1:] - single_mouse['tail_base']['y'].values[:-1]
    
    # speed: assume already included? Add it anyway, JIC
    X['1speed1'] = 0
    X['1speed1'].iloc[1:] = np.sqrt(sx1**2 + sy1**2) * fps

    # mix 'ear_left+nose' and ear_right+nose'
    if 'ear_left+nose' in X.columns:
        X['ear+nose_sum'] = X['ear_left+nose'] + X['ear_right+nose']
        X['ear+nose_diff'] = X['ear_left+nose'] - X['ear_right+nose']
        X.drop(['ear_left+nose','ear_right+nose'], axis=1, inplace=True)

    # drop some features based on FI
    drops = ['s2_20','s2_40','s2_60','s2_80','s3_20','s3_40','s3_60','s3_80']
    drops = list(set(drops).intersection(set(X.columns)))
    X.drop(drops, axis=1, inplace=True)

    X = roll_f(X, fps) # add rolled features      
    return X.astype(np.float32, copy=False)





def transform_pair(mouse_pair, body_parts_tracked, fps, section, meta): # ***************************************************************************
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    # drop both ears: [1]=111, -32
    bp = body_parts_tracked.copy()
    if len(bp) > 4: # do not drop ears if only 4 body parts - impacts section 7 only
        bp.remove('ear_left')
        bp.remove('ear_right')
    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(bp, repeat=2)
        if p1 in avail_A and p2 in avail_B
    })
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(bp, repeat=2)], copy=False)


    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)
        speeds = pd.DataFrame({
            'sp_A': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
            'sp_AB': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
            'sp_B': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = cur - past

    if 'body_center' in avail_A and 'body_center' in avail_B: # body centers **********************************************************
        bp = 'body_center'
        cd = np.sqrt((mouse_pair['A'][bp]['x'] - mouse_pair['B'][bp]['x'])**2 +
                     (mouse_pair['A'][bp]['y'] - mouse_pair['B'][bp]['y'])**2)
        X['cd']   = cd.astype(float)
        cd_full = np.square(mouse_pair['A'][bp] - mouse_pair['B'][bp]).sum(axis=1, skipna=False) # no sqrt here, dist**2
        Axd = mouse_pair['A'][bp]['x'].diff()
        Ayd = mouse_pair['A'][bp]['y'].diff()
        Bxd = mouse_pair['B'][bp]['x'].diff()
        Byd = mouse_pair['B'][bp]['y'].diff()
        coord = Axd * Bxd + Ayd * Byd
        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'd_m{w}']  = cd_full.rolling(ws, **roll).mean()
            X[f'd_s{w}']  = cd_full.rolling(ws, **roll).std()
            X[f'd_mn{w}'] = cd_full.rolling(ws, **roll).min()
            X[f'd_mx{w}'] = cd_full.rolling(ws, **roll).max()

            d_var = cd_full.rolling(ws, **roll).var()
            X[f'int{w}'] = 1 / (1 + d_var)

            X[f'co_m{w}'] = coord.rolling(ws, **roll).mean()
            X[f'co_s{w}'] = coord.rolling(ws, **roll).std()


    if 'nose' in avail_A and 'nose' in avail_B: # noses **********************************************************
        nn = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                     (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nn_lg{lag}']  = nn.shift(l)
            X[f'nn_ch{lag}']  = nn - nn.shift(l)
            is_cl = (nn < 10.0).astype(float)
            X[f'cl_ps{lag}']  = is_cl.rolling(l, min_periods=1).mean()


    if 'body_center' in avail_A and 'body_center' in avail_B: # body centers **********************************************************
        val = (Axd * Bxd + Ayd * Byd) / (np.sqrt(Axd**2 + Ayd**2) * np.sqrt(Bxd**2 + Byd**2) + 1e-6) # cos of angle between speeds
        for off in [-30, -20, -10, 0, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'va_{off}'] = val.shift(-o)
        w = _scale(30, fps)
        X['int_con'] = cd_full.rolling(w, min_periods=1, center=True).std() / \
                       (cd_full.rolling(w, min_periods=1, center=True).mean() + 1e-6)
        X = add_interaction_features(X, mouse_pair, fps, bp)


    # all have both ears and tail base
    # almost all have nose
    # cos/sin of angle between mice, based on ear-to-ear vectors
    ox1 = mouse_pair['A']['ear_left']['x'].values - mouse_pair['A']['ear_right']['x'].values
    ox2 = mouse_pair['B']['ear_left']['x'].values - mouse_pair['B']['ear_right']['x'].values
    oy1 = mouse_pair['A']['ear_left']['y'].values - mouse_pair['A']['ear_right']['y'].values
    oy2 = mouse_pair['B']['ear_left']['y'].values - mouse_pair['B']['ear_right']['y'].values
    osize1 = np.sqrt(ox1**2 + oy1**2)
    osize2 = np.sqrt(ox2**2 + oy2**2)

    # features: angle of each mouse to vector between them
    dx = mouse_pair['A']['tail_base']['x'].values - mouse_pair['B']['tail_base']['x'].values
    dy = mouse_pair['A']['tail_base']['y'].values - mouse_pair['B']['tail_base']['y'].values

    # speed
    # base it on all BODY parts, not just tail_base
    avail = set(mouse_pair['A'].columns.get_level_values(0)).intersection(set(mouse_pair['B'].columns.get_level_values(0)))
    avail = list(avail.intersection(set(['hip_left','hip_right','lateral_left','lateral_right','tail_base','neck','body_center']))) # only use some body parts - 'body' only. No head or tail
    v = np.array([mouse_pair['A'][p]['x'].values for p in avail]).mean(0)
    sx1 = v[1:] - v[:-1]
    v = np.array([mouse_pair['B'][p]['x'].values for p in avail]).mean(0)
    sx2 = v[1:] - v[:-1]
    v = np.array([mouse_pair['A'][p]['y'].values for p in avail]).mean(0)
    sy1 = v[1:] - v[:-1]
    v = np.array([mouse_pair['B'][p]['y'].values for p in avail]).mean(0)
    sy2 = v[1:] - v[:-1]

    # speed
    X['speed1'] = 0
    X['speed1'].iloc[1:] = np.sqrt(sx1**2 + sy1**2) * fps
    X['speed2'] = 0
    X['speed2'].iloc[1:] = np.sqrt(sx2**2 + sy2**2) * fps

    # relative speed
    X['rel_speed'] = 0
    X['rel_speed'].iloc[1:] = np.sqrt((sx1 - sx2)**2 + (sy1 - sy2)**2) * fps

    # cos s1 s2
    X['cos_s1_s2'] = 1
    X['cos_s1_s2'].iloc[1:] = (sx2 * sx1 + sy2 * sy1) / np.sqrt(sx2**2 + sy2**2) / np.sqrt(sx1**2 + sy1**2)

    X = roll_f(X, fps) # add rolled features
    return X.astype(np.float32, copy=False)


## Training, validation and submission
def predict_multiclass(pred, meta, thresholds): # for 1 section only    
    threshold_array = np.array([thresholds.get(col, 0.27) for col in pred.columns]) # for 1 section only, single vs pair. Turn into array by action.
    
    # v3: highest prob/th among actions with p>th
    pred1 = np.concatenate((np.zeros([pred.shape[0], 1], dtype=np.int8), pred.values), axis=1)
    th1 = np.concatenate((np.ones([1], dtype=np.int8), threshold_array))
    ama = np.argmax(pred1 / th1.reshape(1, -1), axis=1)
    max_proba = (pred1 / th1.reshape(1, -1)).max(axis=1)
    ama = np.where(max_proba >= 1, ama - 1, -1) # only keep where it is over threshold; set to -1 otherwise

    # use selected actions to form submission *********************
    ama = pd.Series(ama, index=meta.video_frame) # make 'frame' the final output
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    
    mask = ama_changes.values >= 0 # pos means start of new action, ends are dropped
    mask[-1] = False
    
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })
    
    stop_video_id = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        if stop_video_id[i] != video_id or stop_agent_id[i] != agent_id or stop_target_id[i] != target_id:
            new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame

    return submission_part


# only used to create cat lab feature
lab_di = {'AdaptableSnail': 0, 'BoisterousParrot': 1, 'CautiousGiraffe': 2, 'DeliriousFly': 3, 'ElegantMink': 4, 'GroovyShrew': 5
          , 'InvincibleJellyfish': 6, 'JovialSwallow': 7, 'LyricalHare': 8, 'NiftyGoldfinch': 9, 'PleasantMeerkat': 10
          , 'ReflectiveManatee': 11, 'SparklingTapir': 12, 'TranquilPanther': 13, 'UppityFerret': 14 # real labs: 0-14
          , 'CRIM13': 15, 'CalMS21_supplemental': 16, 'CalMS21_task1': 16, 'CalMS21_task2': 16} # additional training labs only


def find_th(oof_action, y_action):# find best threshold
    a = -oof_action.copy()  # predicted probability
    b = y_action.copy()     # y
    idx = np.argsort(a, order=None)
    a = -a[idx]
    b = b[idx]
    b = b.cumsum()
    f = 2 * b / (b[-1] + np.arange(b.shape[0]))
    j = f.argmax()
    th = np.round(a[j], 3)
    f = f1_score(y_action, (oof_action >= th), zero_division=0)
    return th, f, b[j], b[-1], j # threshold, F1, TP, YC, YPC


def FE(X, meta, sp, section):
    m_di = {'mouse1':0, 'mouse2':1, 'mouse3':2, 'mouse4':3}

    # round to 5 digits
    X = np.round(X, 5)

    # add lab. make it cat. Helps a lot.
    if section in [4, 5, 6, 8]: # only these sections have >1 lab
        X['l'] = meta['lab_id'].map(lab_di).astype('int8').values
        if section == 8: # only section 8 has > 2 labs
            X['l'] = X['l'].astype('category')

    return X



def f1f(labels, preds): # early stopping functon
    preds = preds.astype(np.float32)
    labels = labels.astype(np.int8)
    f1, yc = 0, labels.sum()
    for i in range(1, 50): # test all th in .01 incr from .01 to .49. Best s/b 0.28
        p = (preds > i / 100).astype(np.int8)
        f1 = max(f1, 2 * (p * labels).sum() / (p.sum() + yc))
    return -f1

# XGB paramters ************************************************
n_estimators        = 250 * 4   # make it high enough for ES
learning_rate       = 0.08
max_depth           = 8
min_child_weight    = 15
colsample_bytree    = 0.6
subsample           = 0.8
max_bin             = 32
scale_pos_weight    = 1.1

fi_sum, fi_cnt = {}, {}
names = ['a','b','c','d','e','f','g','h','i','j']
def cross_validate_classifier(sp, X, label, meta, section):
    submission_list, ths, ths2 = [], {}, {}
    
    # drop actions with no targets: happens for section 5 (sniffgenital), 7 (intromit, mount), 2 (sniff, intromit, mount, rear, selfgroom)
    # for section 8, drop approach and sniffface - they only appear in excluded labs!
    for c in label.columns:
        if label[c].max() < 1 or c in ['ejaculate', 'dominancemount', 'genitalgroom', 'disengage'] or (section == 8 and c in ['approach', 'sniffface']):
            label.drop(c, axis=1, inplace=True)
            
    # drop records with no mask: section 5 (18K records), 2 (314K records)
    keep = (label.reset_index(drop=True).fillna(-1).max(1) > -.5).values
    if keep.sum() < len(keep):
        meta  = meta.loc[keep].reset_index(drop=True)
        label = label.loc[keep]
        X     = X.loc[keep]

    oof = pd.DataFrame(index=meta.video_frame)
    X = FE(X, meta, sp, section)
    actions = ['attack','sniff','escape','shepherd','chase','sniffgenital','rear','mount','defend','reciprocalsniff','selfgroom','approach','intromit'
               ,'dominance','huddle','follow','allogroom','dig','attemptmount','sniffbody','avoid','submit','chaseattack','freeze','climb'
               ,'dominancegroom','rest','run','sniffface','flinch','exploreobject','biteobject','tussle']
    for action in [a for a in actions if a in label.columns]: # order by decreasing importance ****
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(np.int8)
        X_action = X[action_mask]
        groups_action = meta.video_id[action_mask]
        oof_action = np.zeros(X_action.shape[0], dtype=np.float32)
        X_action = X_action.reset_index(drop=True)
        groups_action = groups_action.values
        y1 = y_action + 2 * meta['lab_id'].map(lab_di)[action_mask]
        if CFG.mode == 'validate': # training code ******************************************************************************************
            if str(section) + action in best_features: # drop non-selected features
                cols = best_features[str(section) + action]
            else:
                cols = X_action.columns
            X_action = X_action[cols].copy() # break the link and select columns
            kf = StratifiedGroupKFold(n_splits=folds, shuffle=False)
            for fold, (tr_ind, va_ind) in enumerate(kf.split(X_action, y1, groups_action)): # groups by video
                model = XGBClassifier(objective='binary:logistic', verbosity=0, random_state=seed
                    , n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth
                    , min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree
                    , max_bin=max_bin, scale_pos_weight=scale_pos_weight
                    , n_jobs=16, device='cuda', tree_method='hist', enable_categorical=True
                    , early_stopping_rounds=10, eval_metric=f1f)
                model = model.fit(X_action.loc[tr_ind], y_action[tr_ind], eval_set=[(X_action.loc[va_ind], y_action[va_ind])], verbose=False)
                oof_action[va_ind] = model.predict_proba(X_action.loc[va_ind])[:,1]
                model.save_model(CFG.model_name + '/' + str(section) + action + 'xgbmodel' + names[fold] + '.json')  # Saves in JSON format
                del model
                gc.collect()
            # tune smoothing per section/action
            f1b = 0
            for wl in [0, 5, 10, 20, 30, 35, 40, 50, 70, 100, 150, 250]:
                if wl > 0:
                    oa = savgol_filter(oof_action, window_length=wl, polyorder=1, axis=0) # smooth prediction ******************************
                else:
                    oa = oof_action
                threshold, f1, yp, yc, ypc = find_th(oa, y_action) # only place where this is called
                if f1 > f1b:
                    f1b, thresholdb, wlb, ypb, ycb, ypcb = f1, threshold, wl, yp, yc, ypc
                if wl >= 70 and wlb < wl: # break if score stops increasing
                    break
            ths[action], ths2[action] = thresholdb, wlb # save th, wl
            if wlb > 0:
                oof_action = savgol_filter(oof_action, window_length=wlb, polyorder=1, axis=0) # save pred
            if verbose: print('    endfit', int(time()-t0), section, action, np.round(f1b, 3), np.round(thresholdb, 3), wlb, y_action.sum(), X_action.shape[0], X_action.shape[1])
        else: # prediction code - check only, submit is done elsewhere ***********************************************************************
            kf = StratifiedGroupKFold(n_splits=folds, shuffle=False)
            for fold, (tr_ind, va_ind) in enumerate(kf.split(X_action, y1, groups_action)): # groups by video
                model = XGBClassifier()
                model.load_model(CFG.model_name + '/' + str(section) + action + 'xgbmodel' + names[fold] + '.json')
                oof_action[va_ind] = model.predict_proba(X_action.loc[va_ind])[:,1]
                del model
                gc.collect()
            wl = thresholds2[sp][str(section)][action]
            threshold = thresholds[sp][str(section)][action]
            if wl > 0:
                oof_action = savgol_filter(oof_action, window_length=wl, polyorder=1, axis=0) # smooth prediction ************************
            f1 = f1_score(y_action, (oof_action >= threshold), zero_division=0)
            print('    end predict', int(time()-t0), 'sec, section/action/Fa', section, action, np.round(f1, 3), X_action.shape[0], X_action.shape[1])
        
        oof_column = np.zeros(len(label))
        oof_column[action_mask] = oof_action # use this for final pred (smoothed)
        oof[action] = oof_column
        del oof_action, action_mask, X_action, y_action, groups_action
        gc.collect()

    if CFG.mode != 'validate':
        ths = thresholds[sp][str(section)] # need this - it is used in constructing the final prediction!
    submission_part = predict_multiclass(oof, meta, ths) # this creates the submission ***************
    submission_list.append(submission_part)
    return submission_list, ths, ths2, oof, meta, label # return oof+meta+label, JIC



if CFG.mode == "validate":
    thresholds = {"single": {}, "pair": {}}
    thresholds2 = {"single": {}, "pair": {}}
    best_features = joblib.load(f"{CFG.model_name}/best_features.pkl")
else:
    thresholds = joblib.load(f"{CFG.model_name}/thresholds.pkl") # th- not used for blended submission
    best_features = joblib.load(f"{CFG.model_name}/best_features.pkl")
    mp = joblib.load(f"{CFG.model_name}/mp_di2.pkl") # by lab and action - not used for blended submission
    thresholds2  = joblib.load(f"{CFG.model_name}/thresholds2.pkl") # smoothing period - use different ones for different models
    thresholds2b = joblib.load(f"{CFG.model_name2}/thresholds2.pkl")



def submit(body_parts_tracked_str, switch_tr, section, thresholds, thresholds2, thresholds2b):
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]

    test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
    generator = generate_mouse_data(
        test_subset, 
        'test',
        CFG.test_tracking_path,
        generate_single=(switch_tr == 'single'), 
        generate_pair=(switch_tr == 'pair')
    )

    fps_lookup = (
        test_subset[['video_id', 'frames_per_second']]
        .drop_duplicates('video_id')
        .set_index('video_id')['frames_per_second']
        .to_dict()
    )

    submission_list, ooft, metat = [], [], []
    for switch, data, meta, actions in generator:
        if len(actions) == 0: # nothing to predict, skip. Need this with new FE logic
            continue
        fps = _fps_from_meta(meta, fps_lookup, default_fps=30.0)
        if switch == 'single':
            X = transform_single(data, body_parts_tracked, fps, section, meta)
        else:
            X = transform_pair(data, body_parts_tracked, fps, section, meta)

        X = FE(X, meta, switch, section)
        pred = pd.DataFrame(index=meta.video_frame)
        for action in actions:
            # drop excluded actions - only need to do this for train submission, since these are not in real test data (confirmed)
            if action in ['ejaculate', 'dominancemount', 'genitalgroom', 'disengage']:
                continue
            if str(section) + action in best_features: # drop non-selected features
                cols = best_features[str(section) + action]
            else:
                cols = X.columns
            pred[action] = np.zeros(X.shape[0], dtype=np.float32)

            # first XGB prediction
            if os.path.isfile(CFG.model_name + '/' + str(section) + action + 'xgbmodela.json'):
                for fold in range(folds):
                    model = XGBClassifier(device='cuda') # does this work? Don't think so.
                    model.load_model(CFG.model_name + '/' + str(section) + action + 'xgbmodel' + names[fold] + '.json')
                    model.set_params(device='cuda') # or this?
                    pred[action] += model.predict_proba(X[cols])[:,1] / folds # average
                    del model
                    gc.collect()
                wl = thresholds2[action]
                if wl > 0:
                    pred[action] = savgol_filter(pred[action], window_length=wl, polyorder=1, axis=0) # smooth prediction ********************************
            
            # second XGB prediction
            #if os.path.isfile(CFG.model_name2 + '/' + str(section) + action + 'xgbmodela.json'):
            #    p2 = np.zeros(X.shape[0], dtype=np.float32)
            #    for fold in range(folds):
            #        model = XGBClassifier(device='cuda') # does this work? Don't think so.
            #        model.load_model(CFG.model_name2 + '/' + str(section) + action + 'xgbmodel' + names[fold] + '.json')
            #        model.set_params(device='cuda') # or this?
            #        p2 += model.predict_proba(X[cols])[:,1] / folds # average
            #        del model
            #        gc.collect()
            #    wl = thresholds2b[action]
            #    if wl > 0:
            #        p2 = savgol_filter(p2, window_length=wl, polyorder=1, axis=0) # smooth prediction ********************************
            #    # blend the 2 XGB preds
            #    pred[action] = pred[action].values * 0.5 + p2 * 0.5 # 50/50 should do it
                    
        if pred.shape[1] != 0:
            mp2 = mp[meta['lab_id'].iloc[0]]
            submission_part = predict_multiclass(pred, meta, mp2)
            submission_list.append(submission_part)
            ooft.append(pred)
            metat.append(meta)
    return submission_list, ooft, metat


XGB code

In [6]:

submission_list, oofs, metas, labels = [], [], [], []
for section in [8, 6, 3, 4, 1, 9, 5, 7, 2]: # order sections by decreasing importance
        body_parts_tracked_str = body_parts_tracked_list[section]
        body_parts_tracked = json.loads(body_parts_tracked_str)
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
        if verbose: print('section', section)

        if CFG.mode == "submit": # submit logic ***************************************************************************************
            print(int(time()-t0), 'sec')
            if f"{section}" in thresholds["single"].keys(): # skip single actions for some sections
                temp_submission_list, oof, meta = submit(body_parts_tracked_str, 'single', section, thresholds["single"][f"{section}"], thresholds2["single"][f"{section}"], thresholds2b["single"][f"{section}"])
                submission_list.extend(temp_submission_list)
                oofs.extend(oof)
                metas.extend(meta)
            temp_submission_list, oof, meta = submit(body_parts_tracked_str, 'pair', section, thresholds["pair"][f"{section}"], thresholds2["pair"][f"{section}"], thresholds2b["pair"][f"{section}"])
            submission_list.extend(temp_submission_list)
            oofs.extend(oof)
            metas.extend(meta)
            continue

        # the rest is validate/check logic only *****************
        single_mouse_list, single_mouse_label_list, single_mouse_meta_list = [], [], []
        mouse_pair_list, mouse_pair_label_list, mouse_pair_meta_list = [], [], []
        
        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]
        genmode = 'train'
            
        _fps_lookup = (
            train_subset[['video_id', 'frames_per_second']]
            .drop_duplicates('video_id')
            .set_index('video_id')['frames_per_second']
            .to_dict()
        )
        for switch, data, meta, label in generate_mouse_data(train_subset, genmode, CFG.train_tracking_path):
            # drop excluded actions; keep videos with no targets, drop them only later!!
            for c in label.columns:
                if c in ['ejaculate', 'dominancemount', 'genitalgroom', 'disengage']:
                    label.drop(c, axis=1, inplace=True)
                    
            # drop records with no mask
            keep = (label.reset_index(drop=True).fillna(-1).max(1) > -.5).values
            if keep.sum() < len(keep):
                if keep.sum() == 0: # skip empty outputs
                    continue
                meta  = meta.loc[keep].reset_index(drop=True)
                data  = data.loc[keep]
                label = label.loc[keep]
            
            if switch == 'single':
                single_mouse_list.append(data)
                single_mouse_meta_list.append(meta)
                single_mouse_label_list.append(label)
            else:
                mouse_pair_list.append(data)
                mouse_pair_meta_list.append(meta)
                mouse_pair_label_list.append(label)
        del data, meta, label
        gc.collect()

        if len(mouse_pair_list) > 0: # pair *********************************
            pair_feats_parts = []
            for mouse_pair, meta in zip(mouse_pair_list, mouse_pair_meta_list):
                fps = _fps_from_meta(meta, _fps_lookup, default_fps=30.0)
                X = transform_pair(mouse_pair, body_parts_tracked, fps, section, meta).astype(np.float32)
                pair_feats_parts.append(X)
           
            X = pd.concat(pair_feats_parts, axis=0, ignore_index=True)
            label = pd.concat(mouse_pair_label_list, axis=0, ignore_index=True)
            meta = pd.concat(mouse_pair_meta_list, axis=0, ignore_index=True)
            del pair_feats_parts, mouse_pair_list, mouse_pair_label_list, mouse_pair_meta_list
            gc.collect()

            temp_submission_list, temp_thresholds, temp_thresholds2, oof, meta, label = cross_validate_classifier('pair', X, label, meta, section)
                                    
            if CFG.mode == 'validate':
                if f"{section}" not in thresholds["pair"].keys():
                    thresholds["pair"][f"{section}"] = {}
                if f"{section}" not in thresholds2["pair"].keys():
                    thresholds2["pair"][f"{section}"] = {}
                for k, v in temp_thresholds.items():
                    thresholds["pair"][f"{section}"][k] = v
                for k, v in temp_thresholds2.items():
                    thresholds2["pair"][f"{section}"][k] = v
                
            submission_list.extend(temp_submission_list)
            oofs.append(oof)
            metas.append(meta)
            labels.append(label)
            del temp_submission_list, temp_thresholds, temp_thresholds2, X, oof, meta, label
            gc.collect()

        if len(single_mouse_list) > 0: # single *****************************************************
            single_feats_parts = []
            for single_mouse, meta in zip(single_mouse_list, single_mouse_meta_list):
                fps = _fps_from_meta(meta, _fps_lookup, default_fps=30.0)
                X = transform_single(single_mouse, body_parts_tracked, fps, section, meta).astype(np.float32)
                single_feats_parts.append(X)
            
            X = pd.concat(single_feats_parts, axis=0, ignore_index=True)
            label = pd.concat(single_mouse_label_list, axis=0, ignore_index=True)
            meta = pd.concat(single_mouse_meta_list, axis=0, ignore_index=True)
            del single_feats_parts, single_mouse_list, single_mouse_label_list, single_mouse_meta_list
            gc.collect()
            
            temp_submission_list, temp_thresholds, temp_thresholds2, oof, meta, label = cross_validate_classifier('single', X, label, meta, section)
            
            if CFG.mode == 'validate':
                if f"{section}" not in thresholds["single"].keys():
                    thresholds["single"][f"{section}"] = {}
                if f"{section}" not in thresholds2["single"].keys():
                    thresholds2["single"][f"{section}"] = {}
                for k, v in temp_thresholds.items():
                    thresholds["single"][f"{section}"][k] = v
                for k, v in temp_thresholds2.items():
                    thresholds2["single"][f"{section}"][k] = v                 
            
            submission_list.extend(temp_submission_list)
            oofs.append(oof)
            metas.append(meta)
            labels.append(label)
            del temp_submission_list, temp_thresholds, temp_thresholds2, X, oof, meta, label
            gc.collect()

# Validate/check
if CFG.mode != 'submit':
    if CFG.mode == 'validate':
        joblib.dump(thresholds, f"{CFG.model_name}/thresholds.pkl") # these are used for prediction
        joblib.dump(thresholds2, f"{CFG.model_name}/thresholds2.pkl") # these are used for prediction
    submission = pd.concat(submission_list)

    s = np.round(score(solution.loc[(solution['lab_id'].str[:8] != 'CalMS21_') & (solution['lab_id'].str[:6] != 'CRIM13')], submission, ''), 4)    
    if verbose: print('score for selected labs:', s)
    if verbose: print(submission.shape, solution.shape)


# Submission
if CFG.mode == 'submit':
    # construct prediction from totals
    oofs = pd.concat(oofs, axis=0, ignore_index=True).astype('float32').reset_index(drop=True)
    metas = pd.concat(metas, axis=0, ignore_index=True).reset_index(drop=True)
    metas['lab_id'] = metas['lab_id'].map(lab_di).astype('int8')
    idx = metas['target_id'] == 'self'
    metas['target_id'].loc[idx] = metas['agent_id'].loc[idx]
    metas['target_id'] = metas['target_id'].map({'mouse1':0, 'mouse2':1, 'mouse3':2, 'mouse4':3}).astype('int8')
    metas['agent_id']  =  metas['agent_id'].map({'mouse1':0, 'mouse2':1, 'mouse3':2, 'mouse4':3}).astype('int8')
    
    df = pd.concat([metas, oofs], axis=1).reset_index(drop=True)
    del metas, oofs
    gc.collect()
    
    print(df.columns)
    print(df.shape)
    print(df.dtypes)
    print('finished XGB pred')

section 8
0 sec
section 6
0 sec
section 3
0 sec
section 4
0 sec
section 1
0 sec
section 9
113 sec
section 5
113 sec
section 7
113 sec
section 2
113 sec
Index(['lab_id', 'video_id', 'agent_id', 'target_id', 'video_frame', 'rear',
       'approach', 'attack', 'avoid', 'chase', 'chaseattack', 'submit'],
      dtype='object')
(294768, 12)
lab_id            int8
video_id         int64
agent_id          int8
target_id         int8
video_frame      int16
rear           float32
approach       float32
attack         float32
avoid          float32
chase          float32
chaseattack    float32
submit         float32
dtype: object
finished XGB pred


# Now, blend it

In [7]:
mp = joblib.load(model_dir + 'mp_blend_tuned.pkl') # use blended tuned mp - no adjustment needed!
mp3 = joblib.load(model_dir + 'mp3.pkl')





# format xgb prediciton, blend it with NN ************************************************************
id_di3 = {}
for k in id_di.keys(): # id
    id_di3[id_di[k][1]] = k
df['id'] = df['video_id'].map(id_di3).fillna(-1).astype('int16')
df['mid'] = df['agent_id'] * 10 + df['target_id']
df.drop(['lab_id', 'video_id','agent_id','target_id'], axis=1, inplace=True)
cols_yp = [action_di2[i] for i in range(33) if action_di2[i] in df.columns]

df3 = df3.iloc[:yp.shape[0]] # trim
df3['l'] = df3['id'].map(id_di).str[2].values.astype(np.int8) # need this!
lab = df3['l']

# merge, left
df = df3.merge(df, how='left', on=['id', 'mid', 'video_frame'])

# fill-in with yp
# ypx0 is missing 300K records, for which actual target is always zero.
# ypx0 has 300K extra records - mostly frames with some missing body parts
ypx0 = yp.copy() # same shape now
for c in cols_yp:
    i = action_di[c]
    idx = df[c].isna()
    df[c].loc[idx] = yp[idx,i] # complete df - fill missing values with yp
    ypx0[:,i] = df[c].values.astype(np.float32) # put df into ypx0
del df
gc.collect()
print('ypx0', int(time() - t0), 'sec')





# blend NN and XGB *****************
w = 0.57
yp_blended = yp * (1 - w) + ypx0 * w # blend





# transform yp_blended to make it in blocks of n for lab=1.
n = 5 # Good results for n=5, 13, 21. Use 5 here.
dft = df3.loc[df3['l']==1].reset_index(drop=True) # select lab 1 only
if dft.shape[0] > 0:
    dft['block'] = dft['video_frame']//n
    dft['p'] = yp_blended[df3['l']==1, 7] # only action=7 for lab=1
    b = dft.groupby(['id','mid','block'])['p'].max().reset_index() # max is better than mean
    dft = dft.merge(b[['id', 'mid', 'block', 'p']], on=['id','mid','block'], how='left') # adds 'p_y'
    yp_blended[df3['l']==1, 7] = dft['p_y'].fillna(0).values.astype(np.float32)
    mp[1,7] = 0.2942 # need to change the threashold





# construct the submission **************
del yp, ypx0
gc.collect()
sub = prepare_sub(yp_blended, df3, mp, mp3, sm=3, sm1=3)





# save submission
sub.to_csv('submission.csv', index=False)
print('done', sub.shape, int(time() - t0), 'sec')

ypx0 114 sec
done (311, 7) 116 sec
